In [1]:
import math
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm
from config import *
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

## Import

In [2]:
df_douban_inter = pd.read_csv(data_preprocessing + 'douban_inter.csv', dtype={'user_id': str, 'item_id': str, 'rating': int})
df_douban_user_feats = pd.read_csv(data_preprocessing + 'douban_user_features.csv', dtype=str)
df_douban_item_feats = pd.read_csv(data_preprocessing + 'douban_item_features.csv', dtype={'item_id': str, 'author': str, 'publisher': str, 'year': str})
# df_douban_inter

In [3]:
df_movie_inter = pd.read_csv(data_preprocessing + 'movie_inter.csv', dtype={'user_id': str, 'item_id': str, 'rating': int, 'timestamp': int})
df_movie_user_feats = pd.read_csv(data_preprocessing + 'movie_user_features.csv', dtype=str)
df_movie_item_feats = pd.read_csv(data_preprocessing + 'movie_item_features.csv', dtype=str)
# df_movie_inter

In [4]:
df_yelp_inter = pd.read_csv(data_preprocessing + 'yelp_inter.csv', dtype={'user_id': str, 'item_id': str, 'rating': int})
df_yelp_user_feats = pd.read_csv(data_preprocessing + 'yelp_user_features.csv', dtype=str)
df_yelp_item_feats = pd.read_csv(data_preprocessing + 'yelp_item_features.csv', dtype=str)
# df_yelp_inter

## NN-Based

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from deepctr.models import FNN, PNN, CCPM, NFM, WDL, DCN, DeepFM
from deepctr.models import AFM, xDeepFM, DIN
from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [6]:
Kf = KFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
all_result = []

## NN-based

In [7]:
def get_feature_columns(df_all):
    # treat year as categorical
    sparse_features = list(df_all.columns[:-1])

    df_all[sparse_features] = df_all[sparse_features].fillna('-1')

    # 1. label encode
    for feat in sparse_features:
        lbe = LabelEncoder()
        df_all[feat] = lbe.fit_transform(df_all[feat])

    # 2. count #unique features for each sparse field
    fixlen_feature_columns = [SparseFeat(feat, df_all[feat].nunique()) for feat in sparse_features]
    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns
    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
    return linear_feature_columns, dnn_feature_columns, feature_names

In [8]:
def calc_nn_evaluate(df_test, pred, k=10, threshold=3.5):
    rmse = round(np.sqrt(mean_squared_error(df_test['rating'].values, pred)), 4)
    dict_user_ratings = defaultdict(list)
    for i, x in df_test.iterrows():
        dict_user_ratings[x['user_id']].append((pred[i], x['rating']))
    
    ls_recall, ls_ndcg = [], []
    for user_id, ls_user_rating in dict_user_ratings.items():
        ls_user_rating.sort(key=lambda x: x[0], reverse=True)
        ls_true_sort = sorted(ls_user_rating, key=lambda x: x[1], reverse=True)
        # recall
        num_rel = sum(1 for (pred, true_rate) in ls_user_rating if true_rate > threshold)
        num_rel_topk = sum(1 for (pred, true_rate) in ls_user_rating[:k] if true_rate > threshold and pred > threshold)
        recall = num_rel_topk / num_rel if num_rel !=0 else 0
        ls_recall.append(recall)

        # ndcg
        dcg = sum(true_rate / math.log(i + 2, 2) for i, (pred, true_rate) in enumerate(ls_user_rating[:k]))
        idcg = sum(true_rate / math.log(i + 2, 2) for i, (pred, true_rate) in enumerate(ls_true_sort[:k]))
        ndcg = dcg / idcg
        ls_ndcg.append(ndcg)
    
    final_recall = round(sum(ls_recall) / len(ls_recall), 4)
    final_ndcg = round(sum(ls_ndcg) / len(ls_ndcg), 4)
    return [rmse, final_recall, final_ndcg]

In [9]:
def calc_nn_based_model(df_all, 
                        Model, model_params,
                        feature_names,
                        data_name, model_name):
    ls_result = []
    for i, (train_idx, test_idx) in enumerate(Kf.split(df_all)):
        df_train, df_test = df_all.iloc[train_idx].reset_index(drop=True), df_all.iloc[test_idx].reset_index(drop=True)
        train_model_input = {name: df_train[name] for name in feature_names}
        test_model_input = {name: df_test[name] for name in feature_names}
        model = Model(**model_params)
        model.compile('adam', 'mse', metrics=['mse'])
        es = EarlyStopping(monitor='val_mse', mode='min', patience=5, restore_best_weights=True)
        mdckpt = ModelCheckpoint(filepath=f'./saved_model/tf_{model_name}_{data_name}/', monitor='val_mse', save_best_only=True, mode='min')
        history = model.fit(
            train_model_input, df_train['rating'].values, 
            batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_split=0.2,
            callbacks=[es, mdckpt])
        pred = model.predict(test_model_input, batch_size=BATCH_SIZE)
        print('test MSE', round(mean_squared_error(df_test['rating'].values, pred), 4))
        ls_result.append([model_name, i, data_name] + calc_nn_evaluate(df_test, pred))
    return ls_result

### douban

In [10]:
df_douban_all = pd.merge(df_douban_inter, df_douban_user_feats, on='user_id', how='left')
df_douban_all = pd.merge(df_douban_all, df_douban_item_feats, on='item_id', how='left')
df_douban_all['rating'] = df_douban_all.pop('rating')
print(df_douban_all.shape)
df_douban_all.head(3)

(790197, 7)


,user_id,item_id,location,author,publisher,year,rating
0,10855,938,33,4387,944,20,4
1,10027,3,394,9486,1743,14,3
2,741,2426,13,10548,1475,21,5


In [11]:
linear_feature_columns, dnn_feature_columns, feature_names = get_feature_columns(df_douban_all)
df_douban_all.head(3)

,user_id,item_id,location,author,publisher,year,rating
0,848,21659,244,4571,1755,13,4
1,30,14570,313,10236,828,6,3
2,9117,13934,32,612,530,14,5


In [12]:
# FNN
dict_FNN = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                FNN, dict_FNN,
                                feature_names, 
                                'douban', 'FNN')
all_result.extend(ls_result)

Epoch 1/30
1976/1976 [==============================] - 16s 7ms/step - loss: 0.7376 - mse: 0.7363 - val_loss: 0.4997 - val_mse: 0.4972


Epoch 2/30
1976/1976 [==============================] - 13s 7ms/step - loss: 0.4811 - mse: 0.4774 - val_loss: 0.4973 - val_mse: 0.4925


Epoch 3/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4677 - mse: 0.4616 - val_loss: 0.4948 - val_mse: 0.4877


Epoch 4/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4537 - mse: 0.4454 - val_loss: 0.4976 - val_mse: 0.4884
Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4404 - mse: 0.4301 - val_loss: 0.4973 - val_mse: 0.4861


Epoch 6/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4256 - mse: 0.4133 - val_loss: 0.5227 - val_mse: 0.5096
Epoch 7/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4115 - mse: 0.3974 - val_loss: 0.5209 - val_mse: 0.5061
Epoch 8/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3999 - mse: 0.3843 - val_loss: 0.5183 - val_mse: 0.5020
Epoch 9/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3903 - mse: 0.3732 - val_loss: 0.5287 - val_mse: 0.5111
Epoch 10/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3826 - mse: 0.3644 - val_loss: 0.5340 - val_mse: 0.5151
test MSE 0.4903
Epoch 1/30
1976/1976 [==============================] - 16s 7ms/step - loss: 0.7236 - mse: 0.7224 - val_loss: 0.5006 - val_mse: 0.4982


...

Epoch 3/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4625 - mse: 0.4563 - val_loss: 0.4996 - val_mse: 0.4924
Epoch 4/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4488 - mse: 0.4403 - val_loss: 0.4981 - val_mse: 0.4887
Epoch 5/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4356 - mse: 0.4251 - val_loss: 0.5009 - val_mse: 0.4895
Epoch 6/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4227 - mse: 0.4103 - val_loss: 0.5073 - val_mse: 0.4941
Epoch 7/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4109 - mse: 0.3968 - val_loss: 0.5138 - val_mse: 0.4989
test MSE 0.4842
Epoch 1/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.7006 - mse: 0.6994 - val_loss: 0.4985 - val_mse: 0.4961


Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4810 - mse: 0.4773 - val_loss: 0.4936 - val_mse: 0.4889


Epoch 3/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4632 - mse: 0.4571 - val_loss: 0.4931 - val_mse: 0.4860


Epoch 4/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4493 - mse: 0.4409 - val_loss: 0.4916 - val_mse: 0.4822


Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4335 - mse: 0.4229 - val_loss: 0.4993 - val_mse: 0.4878
Epoch 6/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4187 - mse: 0.4062 - val_loss: 0.5102 - val_mse: 0.4968
Epoch 7/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4060 - mse: 0.3917 - val_loss: 0.5181 - val_mse: 0.5030
Epoch 8/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3955 - mse: 0.3796 - val_loss: 0.5272 - val_mse: 0.5106
Epoch 9/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3869 - mse: 0.3696 - val_loss: 0.5339 - val_mse: 0.5160
test MSE 0.4782


In [13]:
# IPNN
dict_IPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_inner': True}
ls_result = calc_nn_based_model(df_douban_all, 
                                PNN, dict_IPNN,
                                feature_names, 
                                'douban', 'IPNN')
all_result.extend(ls_result)

Epoch 1/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.7071 - mse: 0.7060 - val_loss: 0.4965 - val_mse: 0.4944


Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4682 - mse: 0.4650 - val_loss: 0.4852 - val_mse: 0.4810


Epoch 3/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4475 - mse: 0.4421 - val_loss: 0.4852 - val_mse: 0.4787


Epoch 4/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4262 - mse: 0.4186 - val_loss: 0.4996 - val_mse: 0.4910
Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4076 - mse: 0.3980 - val_loss: 0.5037 - val_mse: 0.4931
Epoch 6/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3941 - mse: 0.3826 - val_loss: 0.5158 - val_mse: 0.5035
Epoch 7/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3841 - mse: 0.3710 - val_loss: 0.5221 - val_mse: 0.5083
Epoch 8/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3769 - mse: 0.3624 - val_loss: 0.5326 - val_mse: 0.5174
test MSE 0.4833
Epoch 1/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.7258 - mse: 0.7248 - val_loss: 0.4948 - val_mse: 0.4927


Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4724 - mse: 0.4693 - val_loss: 0.4874 - val_mse: 0.4833


...

Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4703 - mse: 0.4671 - val_loss: 0.4900 - val_mse: 0.4858


Epoch 3/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4516 - mse: 0.4462 - val_loss: 0.4947 - val_mse: 0.4883
Epoch 4/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4327 - mse: 0.4252 - val_loss: 0.4994 - val_mse: 0.4910
Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4163 - mse: 0.4068 - val_loss: 0.5070 - val_mse: 0.4966
Epoch 6/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4013 - mse: 0.3899 - val_loss: 0.5126 - val_mse: 0.5004
Epoch 7/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3894 - mse: 0.3764 - val_loss: 0.5251 - val_mse: 0.5113
test MSE 0.484
Epoch 1/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.7036 - mse: 0.7026 - val_loss: 0.4953 - val_mse: 0.4933


Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4722 - mse: 0.4691 - val_loss: 0.4879 - val_mse: 0.4838


Epoch 3/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4491 - mse: 0.4438 - val_loss: 0.4879 - val_mse: 0.4815


Epoch 4/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4287 - mse: 0.4210 - val_loss: 0.4982 - val_mse: 0.4895
Epoch 5/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4110 - mse: 0.4012 - val_loss: 0.5106 - val_mse: 0.4999
Epoch 6/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.3973 - mse: 0.3857 - val_loss: 0.5165 - val_mse: 0.5040
Epoch 7/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.3880 - mse: 0.3747 - val_loss: 0.5286 - val_mse: 0.5145
Epoch 8/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3807 - mse: 0.3660 - val_loss: 0.5423 - val_mse: 0.5269
test MSE 0.4768


In [14]:
# OPNN
dict_OPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_outter': True}
ls_result = calc_nn_based_model(df_douban_all, 
                                PNN, dict_OPNN,
                                feature_names, 
                                'douban', 'OPNN')
all_result.extend(ls_result)

Epoch 1/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.6947 - mse: 0.6937 - val_loss: 0.4929 - val_mse: 0.4909


Epoch 2/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4696 - mse: 0.4664 - val_loss: 0.4846 - val_mse: 0.4804


Epoch 3/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4448 - mse: 0.4394 - val_loss: 0.5070 - val_mse: 0.5005
Epoch 4/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4228 - mse: 0.4150 - val_loss: 0.5026 - val_mse: 0.4939
Epoch 5/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4055 - mse: 0.3957 - val_loss: 0.5121 - val_mse: 0.5013
Epoch 6/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.3935 - mse: 0.3819 - val_loss: 0.5178 - val_mse: 0.5053
Epoch 7/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.3852 - mse: 0.3720 - val_loss: 0.5291 - val_mse: 0.5151
test MSE 0.4842
Epoch 1/30
1976/1976 [==============================] - 16s 7ms/step - loss: 0.7036 - mse: 0.7026 - val_loss: 0.4941 - val_mse: 0.4921


Epoch 2/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4680 - mse: 0.4649 - val_loss: 0.4955 - val_mse: 0.4914


Epoch 3/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4477 - mse: 0.4423 - val_loss: 0.4876 - val_mse: 0.4813


...

Epoch 2/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4695 - mse: 0.4663 - val_loss: 0.4855 - val_mse: 0.4813


Epoch 3/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4501 - mse: 0.4448 - val_loss: 0.4887 - val_mse: 0.4824
Epoch 4/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4310 - mse: 0.4236 - val_loss: 0.4935 - val_mse: 0.4852
Epoch 5/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4148 - mse: 0.4054 - val_loss: 0.5045 - val_mse: 0.4943
Epoch 6/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4021 - mse: 0.3908 - val_loss: 0.5093 - val_mse: 0.4972
Epoch 7/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.3914 - mse: 0.3785 - val_loss: 0.5186 - val_mse: 0.5050
test MSE 0.4793
Epoch 1/30
1976/1976 [==============================] - 14s 6ms/step - loss: 0.6927 - mse: 0.6917 - val_loss: 0.4980 - val_mse: 0.4960


Epoch 2/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4738 - mse: 0.4707 - val_loss: 0.4877 - val_mse: 0.4835


Epoch 3/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4517 - mse: 0.4463 - val_loss: 0.4882 - val_mse: 0.4818


Epoch 4/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4323 - mse: 0.4247 - val_loss: 0.4929 - val_mse: 0.4844
Epoch 5/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4136 - mse: 0.4040 - val_loss: 0.5026 - val_mse: 0.4921
Epoch 6/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.3998 - mse: 0.3883 - val_loss: 0.5154 - val_mse: 0.5031
Epoch 7/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.3894 - mse: 0.3763 - val_loss: 0.5443 - val_mse: 0.5305
Epoch 8/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.3813 - mse: 0.3668 - val_loss: 0.5349 - val_mse: 0.5197
test MSE 0.4795


In [15]:
# PIN

In [16]:
# CCPM
dict_CCPM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                CCPM, dict_CCPM,
                                feature_names, 
                                'douban', 'CCPM')
all_result.extend(ls_result)

Epoch 1/30
1976/1976 [==============================] - 27s 12ms/step - loss: 0.7650 - mse: 0.7633 - val_loss: 0.5090 - val_mse: 0.5056


Epoch 2/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4831 - mse: 0.4782 - val_loss: 0.5054 - val_mse: 0.4992


Epoch 3/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4651 - mse: 0.4574 - val_loss: 0.5001 - val_mse: 0.4911


Epoch 4/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4513 - mse: 0.4409 - val_loss: 0.5085 - val_mse: 0.4970
Epoch 5/30
1976/1976 [==============================] - 25s 12ms/step - loss: 0.4376 - mse: 0.4247 - val_loss: 0.5118 - val_mse: 0.4979
Epoch 6/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4252 - mse: 0.4102 - val_loss: 0.5216 - val_mse: 0.5057
Epoch 7/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4147 - mse: 0.3979 - val_loss: 0.5277 - val_mse: 0.5101
Epoch 8/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4060 - mse: 0.3876 - val_loss: 0.5412 - val_mse: 0.5220
test MSE 0.4945
Epoch 1/30
1976/1976 [==============================] - 25s 12ms/step - loss: 0.7230 - mse: 0.7213 - val_loss: 0.5023 - val_mse: 0.4990


Epoch 2/30
1976/1976 [==============================] - 25s 12ms/step - loss: 0.4808 - mse: 0.4760 - val_loss: 0.5088 - val_mse: 0.5028
Epoch 3/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4695 - mse: 0.4622 - val_loss: 0.5093 - val_mse: 0.5009
Epoch 4/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4613 - mse: 0.4517 - val_loss: 0.5023 - val_mse: 0.4918


...

Epoch 5/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4543 - mse: 0.4434 - val_loss: 0.4971 - val_mse: 0.4853


Epoch 6/30
1976/1976 [==============================] - 23s 12ms/step - loss: 0.4440 - mse: 0.4311 - val_loss: 0.5125 - val_mse: 0.4989
Epoch 7/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4330 - mse: 0.4184 - val_loss: 0.5147 - val_mse: 0.4994
Epoch 8/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4225 - mse: 0.4063 - val_loss: 0.5158 - val_mse: 0.4989
Epoch 9/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4142 - mse: 0.3965 - val_loss: 0.5284 - val_mse: 0.5100
Epoch 10/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4066 - mse: 0.3875 - val_loss: 0.5328 - val_mse: 0.5130
test MSE 0.4861
Epoch 1/30
1976/1976 [==============================] - 25s 12ms/step - loss: 0.7286 - mse: 0.7270 - val_loss: 0.4982 - val_mse: 0.4952


Epoch 2/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4781 - mse: 0.4737 - val_loss: 0.4986 - val_mse: 0.4930


Epoch 3/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4627 - mse: 0.4557 - val_loss: 0.4993 - val_mse: 0.4911


Epoch 4/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4465 - mse: 0.4370 - val_loss: 0.5100 - val_mse: 0.4994
Epoch 5/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4331 - mse: 0.4214 - val_loss: 0.5109 - val_mse: 0.4982
Epoch 6/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4215 - mse: 0.4078 - val_loss: 0.5171 - val_mse: 0.5027
Epoch 7/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4123 - mse: 0.3970 - val_loss: 0.5217 - val_mse: 0.5056
Epoch 8/30
1976/1976 [==============================] - 24s 12ms/step - loss: 0.4042 - mse: 0.3873 - val_loss: 0.5274 - val_mse: 0.5099
test MSE 0.4867


In [17]:
# WD
dict_WD = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                WDL, dict_WD,
                                feature_names, 
                                'douban', 'WD')
all_result.extend(ls_result)

Epoch 1/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.6769 - mse: 0.6756 - val_loss: 0.5266 - val_mse: 0.5242


Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4777 - mse: 0.4740 - val_loss: 0.5042 - val_mse: 0.4994


Epoch 3/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4602 - mse: 0.4540 - val_loss: 0.4931 - val_mse: 0.4859


Epoch 4/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4470 - mse: 0.4385 - val_loss: 0.4929 - val_mse: 0.4835


Epoch 5/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4337 - mse: 0.4232 - val_loss: 0.4986 - val_mse: 0.4872
Epoch 6/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4186 - mse: 0.4062 - val_loss: 0.5061 - val_mse: 0.4929
Epoch 7/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4062 - mse: 0.3920 - val_loss: 0.5135 - val_mse: 0.4986
Epoch 8/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.3954 - mse: 0.3797 - val_loss: 0.5217 - val_mse: 0.5053
Epoch 9/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.3859 - mse: 0.3688 - val_loss: 0.5352 - val_mse: 0.5175
test MSE 0.4878
Epoch 1/30
1976/1976 [==============================] - 13s 6ms/step - loss: 0.7196 - mse: 0.7183 - val_loss: 0.4982 - val_mse: 0.4958


...

Epoch 3/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4651 - mse: 0.4590 - val_loss: 0.4874 - val_mse: 0.4803


Epoch 4/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4502 - mse: 0.4419 - val_loss: 0.4946 - val_mse: 0.4854
Epoch 5/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4355 - mse: 0.4250 - val_loss: 0.4959 - val_mse: 0.4845
Epoch 6/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4204 - mse: 0.4080 - val_loss: 0.5066 - val_mse: 0.4933
Epoch 7/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4074 - mse: 0.3933 - val_loss: 0.5229 - val_mse: 0.5080
Epoch 8/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.3965 - mse: 0.3807 - val_loss: 0.5246 - val_mse: 0.5082
test MSE 0.4785
Epoch 1/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.7142 - mse: 0.7130 - val_loss: 0.5110 - val_mse: 0.5086


Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4805 - mse: 0.4768 - val_loss: 0.4908 - val_mse: 0.4860


Epoch 3/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4634 - mse: 0.4573 - val_loss: 0.4885 - val_mse: 0.4814


Epoch 4/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4491 - mse: 0.4407 - val_loss: 0.4941 - val_mse: 0.4848
Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4338 - mse: 0.4233 - val_loss: 0.5006 - val_mse: 0.4892
Epoch 6/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4191 - mse: 0.4066 - val_loss: 0.5076 - val_mse: 0.4943
Epoch 7/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4059 - mse: 0.3917 - val_loss: 0.5152 - val_mse: 0.5002
Epoch 8/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.3965 - mse: 0.3807 - val_loss: 0.5272 - val_mse: 0.5107
test MSE 0.4782


In [18]:
# DC
dict_DC = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                DCN, dict_DC,
                                feature_names, 
                                'douban', 'DC')
all_result.extend(ls_result)

CrossNet parameterization: vector
Epoch 1/30
1976/1976 [==============================] - 19s 9ms/step - loss: 0.6970 - mse: 0.6957 - val_loss: 0.4982 - val_mse: 0.4957


Epoch 2/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4774 - mse: 0.4736 - val_loss: 0.4960 - val_mse: 0.4911


Epoch 3/30
1976/1976 [==============================] - 18s 9ms/step - loss: 0.4609 - mse: 0.4547 - val_loss: 0.4891 - val_mse: 0.4818


Epoch 4/30
1976/1976 [==============================] - 18s 9ms/step - loss: 0.4479 - mse: 0.4394 - val_loss: 0.4905 - val_mse: 0.4810


Epoch 5/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4331 - mse: 0.4225 - val_loss: 0.4990 - val_mse: 0.4874
Epoch 6/30
1976/1976 [==============================] - 18s 9ms/step - loss: 0.4172 - mse: 0.4046 - val_loss: 0.5031 - val_mse: 0.4897
Epoch 7/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4042 - mse: 0.3898 - val_loss: 0.5127 - val_mse: 0.4975
Epoch 8/30
1976/1976 [==============================] - 18s 9ms/step - loss: 0.3933 - mse: 0.3773 - val_loss: 0.5225 - val_mse: 0.5059
Epoch 9/30
1976/1976 [==============================] - 18s 9ms/step - loss: 0.3841 - mse: 0.3668 - val_loss: 0.5260 - val_mse: 0.5081
test MSE 0.4852
CrossNet parameterization: vector
Epoch 1/30
1976/1976 [==============================] - 18s 8ms/step - loss: 0.7098 - mse: 0.7085 - val_loss: 0.5046 - val_mse: 0.5022


...

Epoch 5/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4376 - mse: 0.4271 - val_loss: 0.4985 - val_mse: 0.4871


Epoch 6/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4247 - mse: 0.4123 - val_loss: 0.5157 - val_mse: 0.5025
Epoch 7/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4124 - mse: 0.3983 - val_loss: 0.5126 - val_mse: 0.4977
Epoch 8/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4007 - mse: 0.3850 - val_loss: 0.5222 - val_mse: 0.5058
Epoch 9/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.3912 - mse: 0.3741 - val_loss: 0.5340 - val_mse: 0.5163
Epoch 10/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.3825 - mse: 0.3642 - val_loss: 0.5421 - val_mse: 0.5232
test MSE 0.4865
CrossNet parameterization: vector
Epoch 1/30
1976/1976 [==============================] - 17s 8ms/step - loss: 0.6868 - mse: 0.6855 - val_loss: 0.5017 - val_mse: 0.4993


Epoch 2/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4800 - mse: 0.4762 - val_loss: 0.4900 - val_mse: 0.4851


Epoch 3/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4624 - mse: 0.4562 - val_loss: 0.4939 - val_mse: 0.4867
Epoch 4/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4505 - mse: 0.4420 - val_loss: 0.4942 - val_mse: 0.4848


Epoch 5/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4366 - mse: 0.4260 - val_loss: 0.4979 - val_mse: 0.4865
Epoch 6/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4231 - mse: 0.4107 - val_loss: 0.5045 - val_mse: 0.4913
Epoch 7/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4096 - mse: 0.3954 - val_loss: 0.5126 - val_mse: 0.4977
Epoch 8/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.3989 - mse: 0.3832 - val_loss: 0.5200 - val_mse: 0.5036
Epoch 9/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.3893 - mse: 0.3722 - val_loss: 0.5329 - val_mse: 0.5151
test MSE 0.4822


In [19]:
# NFM
dict_NFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                NFM, dict_NFM,
                                feature_names, 
                                'douban', 'NFM')
all_result.extend(ls_result)

Epoch 1/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.6961 - mse: 0.6943 - val_loss: 0.5045 - val_mse: 0.5012


Epoch 2/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4669 - mse: 0.4620 - val_loss: 0.5144 - val_mse: 0.5082
Epoch 3/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4459 - mse: 0.4381 - val_loss: 0.4975 - val_mse: 0.4886


Epoch 4/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4315 - mse: 0.4211 - val_loss: 0.5137 - val_mse: 0.5023
Epoch 5/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4185 - mse: 0.4060 - val_loss: 0.5129 - val_mse: 0.4994
Epoch 6/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4080 - mse: 0.3935 - val_loss: 0.5123 - val_mse: 0.4970
Epoch 7/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4000 - mse: 0.3838 - val_loss: 0.5254 - val_mse: 0.5085
Epoch 8/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.3939 - mse: 0.3762 - val_loss: 0.5348 - val_mse: 0.5164
test MSE 0.4929
Epoch 1/30
1976/1976 [==============================] - 16s 7ms/step - loss: 0.7098 - mse: 0.7080 - val_loss: 0.4952 - val_mse: 0.4918


Epoch 2/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4688 - mse: 0.4638 - val_loss: 0.4966 - val_mse: 0.4904


Epoch 3/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4501 - mse: 0.4423 - val_loss: 0.4988 - val_mse: 0.4899


...

Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4295 - mse: 0.4175 - val_loss: 0.5072 - val_mse: 0.4943
Epoch 6/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4184 - mse: 0.4045 - val_loss: 0.5112 - val_mse: 0.4965
Epoch 7/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4094 - mse: 0.3938 - val_loss: 0.5198 - val_mse: 0.5035
Epoch 8/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4020 - mse: 0.3848 - val_loss: 0.5281 - val_mse: 0.5103
Epoch 9/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.3958 - mse: 0.3773 - val_loss: 0.5527 - val_mse: 0.5336
test MSE 0.4846
Epoch 1/30
1976/1976 [==============================] - 13s 6ms/step - loss: 0.7194 - mse: 0.7176 - val_loss: 0.5029 - val_mse: 0.4994


Epoch 2/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4679 - mse: 0.4629 - val_loss: 0.4912 - val_mse: 0.4848


Epoch 3/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4476 - mse: 0.4397 - val_loss: 0.4953 - val_mse: 0.4863
Epoch 4/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4334 - mse: 0.4230 - val_loss: 0.5038 - val_mse: 0.4924
Epoch 5/30
1976/1976 [==============================] - 14s 7ms/step - loss: 0.4217 - mse: 0.4091 - val_loss: 0.5110 - val_mse: 0.4974
Epoch 6/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4114 - mse: 0.3969 - val_loss: 0.5135 - val_mse: 0.4981
Epoch 7/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.4037 - mse: 0.3874 - val_loss: 0.5359 - val_mse: 0.5189
test MSE 0.4835
Epoch 1/30
1976/1976 [==============================] - 17s 8ms/step - loss: 0.7058 - mse: 0.7039 - val_loss: 0.4985 - val_mse: 0.4951


Epoch 2/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4716 - mse: 0.4667 - val_loss: 0.5024 - val_mse: 0.4962
Epoch 3/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4553 - mse: 0.4478 - val_loss: 0.4937 - val_mse: 0.4851


Epoch 4/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4407 - mse: 0.4308 - val_loss: 0.4963 - val_mse: 0.4855
Epoch 5/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4260 - mse: 0.4140 - val_loss: 0.5020 - val_mse: 0.4891
Epoch 6/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4130 - mse: 0.3990 - val_loss: 0.5102 - val_mse: 0.4954
Epoch 7/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.4034 - mse: 0.3877 - val_loss: 0.5251 - val_mse: 0.5086
Epoch 8/30
1976/1976 [==============================] - 15s 8ms/step - loss: 0.3967 - mse: 0.3794 - val_loss: 0.5270 - val_mse: 0.5090
test MSE 0.4804


In [20]:
# deepFM
dict_deepFM = {'linear_feature_columns': linear_feature_columns, 
               'dnn_feature_columns': dnn_feature_columns,
               'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                DeepFM, dict_deepFM,
                                feature_names, 
                                'douban', 'deepFM')
all_result.extend(ls_result)

Epoch 1/30
1976/1976 [==============================] - 16s 7ms/step - loss: 0.6982 - mse: 0.6969 - val_loss: 0.4978 - val_mse: 0.4954


Epoch 2/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4768 - mse: 0.4731 - val_loss: 0.4896 - val_mse: 0.4847


Epoch 3/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4590 - mse: 0.4528 - val_loss: 0.5038 - val_mse: 0.4966
Epoch 4/30
1976/1976 [==============================] - 17s 8ms/step - loss: 0.4435 - mse: 0.4350 - val_loss: 0.5037 - val_mse: 0.4942
Epoch 5/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4279 - mse: 0.4173 - val_loss: 0.5040 - val_mse: 0.4925
Epoch 6/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4143 - mse: 0.4017 - val_loss: 0.5125 - val_mse: 0.4990
Epoch 7/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4017 - mse: 0.3873 - val_loss: 0.5208 - val_mse: 0.5056
test MSE 0.4888
Epoch 1/30
1976/1976 [==============================] - 15s 7ms/step - loss: 0.7019 - mse: 0.7007 - val_loss: 0.5009 - val_mse: 0.4986


Epoch 2/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4804 - mse: 0.4768 - val_loss: 0.5065 - val_mse: 0.5017
Epoch 3/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4633 - mse: 0.4572 - val_loss: 0.4915 - val_mse: 0.4843


Epoch 4/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4472 - mse: 0.4388 - val_loss: 0.4927 - val_mse: 0.4834


...

Epoch 3/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4634 - mse: 0.4572 - val_loss: 0.4916 - val_mse: 0.4843
Epoch 4/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4483 - mse: 0.4397 - val_loss: 0.4923 - val_mse: 0.4828


Epoch 5/30
1976/1976 [==============================] - 17s 8ms/step - loss: 0.4332 - mse: 0.4225 - val_loss: 0.4964 - val_mse: 0.4848
Epoch 6/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4197 - mse: 0.4070 - val_loss: 0.5107 - val_mse: 0.4972
Epoch 7/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4068 - mse: 0.3923 - val_loss: 0.5166 - val_mse: 0.5013
Epoch 8/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.3964 - mse: 0.3804 - val_loss: 0.5345 - val_mse: 0.5178
Epoch 9/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.3874 - mse: 0.3700 - val_loss: 0.5348 - val_mse: 0.5167
test MSE 0.4821
Epoch 1/30
1976/1976 [==============================] - 19s 8ms/step - loss: 0.7182 - mse: 0.7170 - val_loss: 0.5001 - val_mse: 0.4977


Epoch 2/30
1976/1976 [==============================] - 17s 8ms/step - loss: 0.4795 - mse: 0.4758 - val_loss: 0.4927 - val_mse: 0.4879


Epoch 3/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4617 - mse: 0.4555 - val_loss: 0.4902 - val_mse: 0.4828


Epoch 4/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4456 - mse: 0.4370 - val_loss: 0.5023 - val_mse: 0.4926
Epoch 5/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4298 - mse: 0.4189 - val_loss: 0.5006 - val_mse: 0.4888
Epoch 6/30
1976/1976 [==============================] - 17s 8ms/step - loss: 0.4154 - mse: 0.4026 - val_loss: 0.5133 - val_mse: 0.4997
Epoch 7/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4041 - mse: 0.3896 - val_loss: 0.5224 - val_mse: 0.5071
Epoch 8/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.3946 - mse: 0.3785 - val_loss: 0.5322 - val_mse: 0.5153
test MSE 0.4789


In [21]:
# AFM
dict_AFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                AFM, dict_AFM,
                                feature_names, 
                                'douban', 'AFM')
all_result.extend(ls_result)

Epoch 1/30
1976/1976 [==============================] - 16s 7ms/step - loss: 2.6346 - mse: 2.6284 - val_loss: 0.6233 - val_mse: 0.6143


Epoch 2/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.5636 - mse: 0.5527 - val_loss: 0.5419 - val_mse: 0.5291


Epoch 3/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.5109 - mse: 0.4962 - val_loss: 0.5199 - val_mse: 0.5035


Epoch 4/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4919 - mse: 0.4741 - val_loss: 0.5140 - val_mse: 0.4950


Epoch 5/30
1976/1976 [==============================] - 16s 8ms/step - loss: 0.4839 - mse: 0.4639 - val_loss: 0.5116 - val_mse: 0.4908


...

Epoch 18/30
1976/1976 [==============================] - 18s 9ms/step - loss: 0.4731 - mse: 0.4507 - val_loss: 0.5094 - val_mse: 0.4872
Epoch 19/30
1976/1976 [==============================] - 18s 9ms/step - loss: 0.4727 - mse: 0.4505 - val_loss: 0.5096 - val_mse: 0.4874
Epoch 20/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4726 - mse: 0.4504 - val_loss: 0.5089 - val_mse: 0.4869


Epoch 21/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4723 - mse: 0.4503 - val_loss: 0.5094 - val_mse: 0.4875
Epoch 22/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4720 - mse: 0.4501 - val_loss: 0.5089 - val_mse: 0.4871
Epoch 23/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4718 - mse: 0.4501 - val_loss: 0.5085 - val_mse: 0.4869
Epoch 24/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4715 - mse: 0.4499 - val_loss: 0.5084 - val_mse: 0.4868


Epoch 25/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4713 - mse: 0.4497 - val_loss: 0.5089 - val_mse: 0.4875
Epoch 26/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4712 - mse: 0.4498 - val_loss: 0.5078 - val_mse: 0.4865


Epoch 27/30
1976/1976 [==============================] - 17s 8ms/step - loss: 0.4709 - mse: 0.4496 - val_loss: 0.5080 - val_mse: 0.4868
Epoch 28/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4707 - mse: 0.4495 - val_loss: 0.5078 - val_mse: 0.4868
Epoch 29/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4705 - mse: 0.4494 - val_loss: 0.5071 - val_mse: 0.4861


Epoch 30/30
1976/1976 [==============================] - 17s 9ms/step - loss: 0.4703 - mse: 0.4493 - val_loss: 0.5071 - val_mse: 0.4862
test MSE 0.4828


In [22]:
# xDeepFM
dict_xDeepFM = {'linear_feature_columns': linear_feature_columns,
                'dnn_feature_columns': dnn_feature_columns,
                'task': 'regression'}
ls_result = calc_nn_based_model(df_douban_all, 
                                xDeepFM, dict_xDeepFM,
                                feature_names, 
                                'douban', 'xDeepFM')
all_result.extend(ls_result)

Epoch 1/30
1976/1976 [==============================] - 21s 9ms/step - loss: 0.6675 - mse: 0.6662 - val_loss: 0.4996 - val_mse: 0.4970


Epoch 2/30
1976/1976 [==============================] - 19s 10ms/step - loss: 0.4764 - mse: 0.4725 - val_loss: 0.4888 - val_mse: 0.4837


Epoch 3/30
1976/1976 [==============================] - 20s 10ms/step - loss: 0.4566 - mse: 0.4502 - val_loss: 0.4895 - val_mse: 0.4819


Epoch 4/30
1976/1976 [==============================] - 20s 10ms/step - loss: 0.4385 - mse: 0.4296 - val_loss: 0.4897 - val_mse: 0.4798


Epoch 5/30
1976/1976 [==============================] - 19s 10ms/step - loss: 0.4190 - mse: 0.4078 - val_loss: 0.4963 - val_mse: 0.4841
Epoch 6/30
1976/1976 [==============================] - 20s 10ms/step - loss: 0.4023 - mse: 0.3890 - val_loss: 0.5151 - val_mse: 0.5008
Epoch 7/30
1976/1976 [==============================] - 20s 10ms/step - loss: 0.3892 - mse: 0.3740 - val_loss: 0.5230 - val_mse: 0.5070
Epoch 8/30
1976/1976 [==============================] - 20s 10ms/step - loss: 0.3786 - mse: 0.3619 - val_loss: 0.5358 - val_mse: 0.5183
Epoch 9/30
1976/1976 [==============================] - 20s 10ms/step - loss: 0.3706 - mse: 0.3524 - val_loss: 0.5413 - val_mse: 0.5224
test MSE 0.4842
Epoch 1/30
1976/1976 [==============================] - 21s 10ms/step - loss: 0.6488 - mse: 0.6474 - val_loss: 0.5040 - val_mse: 0.5014


...

Epoch 3/30
1976/1976 [==============================] - 19s 10ms/step - loss: 0.4594 - mse: 0.4529 - val_loss: 0.4870 - val_mse: 0.4794


Epoch 4/30
1976/1976 [==============================] - 19s 10ms/step - loss: 0.4425 - mse: 0.4336 - val_loss: 0.4928 - val_mse: 0.4829
Epoch 5/30
1976/1976 [==============================] - 19s 10ms/step - loss: 0.4268 - mse: 0.4156 - val_loss: 0.4982 - val_mse: 0.4861
Epoch 6/30
1976/1976 [==============================] - 19s 10ms/step - loss: 0.4104 - mse: 0.3972 - val_loss: 0.5074 - val_mse: 0.4934
Epoch 7/30
1976/1976 [==============================] - 19s 9ms/step - loss: 0.3949 - mse: 0.3799 - val_loss: 0.5248 - val_mse: 0.5090
Epoch 8/30
1976/1976 [==============================] - 19s 10ms/step - loss: 0.3828 - mse: 0.3661 - val_loss: 0.5335 - val_mse: 0.5160
test MSE 0.4787
Epoch 1/30
1976/1976 [==============================] - 20s 9ms/step - loss: 0.6585 - mse: 0.6572 - val_loss: 0.5126 - val_mse: 0.5101


Epoch 2/30
1976/1976 [==============================] - 20s 10ms/step - loss: 0.4802 - mse: 0.4764 - val_loss: 0.4902 - val_mse: 0.4851


Epoch 3/30
1976/1976 [==============================] - 20s 10ms/step - loss: 0.4593 - mse: 0.4528 - val_loss: 0.4943 - val_mse: 0.4867
Epoch 4/30
1976/1976 [==============================] - 21s 11ms/step - loss: 0.4382 - mse: 0.4292 - val_loss: 0.4937 - val_mse: 0.4836


Epoch 5/30
1976/1976 [==============================] - 20s 10ms/step - loss: 0.4187 - mse: 0.4073 - val_loss: 0.5021 - val_mse: 0.4897
Epoch 6/30
1976/1976 [==============================] - 20s 10ms/step - loss: 0.4022 - mse: 0.3887 - val_loss: 0.5158 - val_mse: 0.5014
Epoch 7/30
1976/1976 [==============================] - 21s 10ms/step - loss: 0.3893 - mse: 0.3740 - val_loss: 0.5339 - val_mse: 0.5176
Epoch 8/30
1976/1976 [==============================] - 20s 10ms/step - loss: 0.3789 - mse: 0.3618 - val_loss: 0.5423 - val_mse: 0.5244
Epoch 9/30
1976/1976 [==============================] - 21s 10ms/step - loss: 0.3715 - mse: 0.3529 - val_loss: 0.5469 - val_mse: 0.5277
test MSE 0.4797


In [23]:
# DIN
# https://github.com/shenweichen/DeepCTR/blob/master/examples/run_din.py
# dict_DIN = {'dnn_feature_columns': dnn_feature_columns,
#             'history_feature_list': None,
#             'task': 'regression'}
# ls_result = calc_nn_based_model(df_douban_all, 
#                                 DIN, dict_DIN,
#                                 feature_names, 
#                                 'douban', 'DIN')
# all_result.extend(ls_result)

### movie lens

In [24]:
df_movie_all = pd.merge(df_movie_inter, df_movie_user_feats, on='user_id', how='left')
df_movie_all = pd.merge(df_movie_all, df_movie_item_feats, on='item_id', how='left')
df_movie_all['rating'] = df_movie_all.pop('rating')
df_movie_all.iloc[:, :-1] = df_movie_all.iloc[:, :-1].fillna('0')
sr_movie_ts = df_movie_all.pop('timestamp')
print(df_movie_all.shape)
df_movie_all.head(3)

(100000, 23)


,user_id,item_id,age,occupation,genre1,genre10,genre11,genre12,genre13,genre14,...,genre18,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,rating
0,196,242,5,3,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,3
1,186,302,4,4,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,3
2,22,377,3,3,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1


In [25]:
linear_feature_columns, dnn_feature_columns, feature_names = get_feature_columns(df_movie_all)
df_movie_all.head(3)

,user_id,item_id,age,occupation,genre1,genre10,genre11,genre12,genre13,genre14,...,genre18,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,rating
0,107,842,4,14,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,3
1,96,909,3,15,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,3
2,134,991,2,14,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1


In [26]:
# FNN
dict_FNN = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                FNN, dict_FNN,
                                feature_names, 
                                'movie_lens', 'FNN')
all_result.extend(ls_result)

Epoch 1/30
250/250 [==============================] - 8s 15ms/step - loss: 1.8902 - mse: 1.8902 - val_loss: 0.9044 - val_mse: 0.9043


Epoch 2/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8803 - mse: 0.8802 - val_loss: 0.8873 - val_mse: 0.8871


Epoch 3/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8664 - mse: 0.8662 - val_loss: 0.8841 - val_mse: 0.8840


Epoch 4/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8614 - mse: 0.8613 - val_loss: 0.8954 - val_mse: 0.8952
Epoch 5/30
250/250 [==============================] - 4s 14ms/step - loss: 0.8561 - mse: 0.8559 - val_loss: 0.8808 - val_mse: 0.8805


Epoch 6/30
250/250 [==============================] - 4s 14ms/step - loss: 0.8517 - mse: 0.8515 - val_loss: 0.8803 - val_mse: 0.8801


...

Epoch 7/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8360 - mse: 0.8358 - val_loss: 0.8714 - val_mse: 0.8711
Epoch 8/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8266 - mse: 0.8263 - val_loss: 0.8576 - val_mse: 0.8573


Epoch 9/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8188 - mse: 0.8185 - val_loss: 0.8590 - val_mse: 0.8586
Epoch 10/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8087 - mse: 0.8083 - val_loss: 0.8575 - val_mse: 0.8571


Epoch 11/30
250/250 [==============================] - 3s 14ms/step - loss: 0.7980 - mse: 0.7975 - val_loss: 0.8575 - val_mse: 0.8571
Epoch 12/30
250/250 [==============================] - 3s 14ms/step - loss: 0.7922 - mse: 0.7918 - val_loss: 0.8543 - val_mse: 0.8539


Epoch 13/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7828 - mse: 0.7823 - val_loss: 0.8572 - val_mse: 0.8567
Epoch 14/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7714 - mse: 0.7709 - val_loss: 0.8527 - val_mse: 0.8522


Epoch 15/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7635 - mse: 0.7629 - val_loss: 0.8633 - val_mse: 0.8627
Epoch 16/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7534 - mse: 0.7528 - val_loss: 0.8596 - val_mse: 0.8589
Epoch 17/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7459 - mse: 0.7453 - val_loss: 0.8811 - val_mse: 0.8804
Epoch 18/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7366 - mse: 0.7359 - val_loss: 0.8661 - val_mse: 0.8654
Epoch 19/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7282 - mse: 0.7274 - val_loss: 0.8666 - val_mse: 0.8658
test MSE 0.8737


In [27]:
# IPNN
dict_IPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_inner': True}
ls_result = calc_nn_based_model(df_movie_all, 
                                PNN, dict_IPNN,
                                feature_names, 
                                'movie_lens', 'IPNN')
all_result.extend(ls_result)

Epoch 1/30
250/250 [==============================] - 7s 13ms/step - loss: 1.9467 - mse: 1.9467 - val_loss: 0.9181 - val_mse: 0.9180


Epoch 2/30
250/250 [==============================] - 3s 12ms/step - loss: 0.8843 - mse: 0.8842 - val_loss: 0.8864 - val_mse: 0.8863


Epoch 3/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8682 - mse: 0.8681 - val_loss: 0.8845 - val_mse: 0.8844


Epoch 4/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8598 - mse: 0.8596 - val_loss: 0.8912 - val_mse: 0.8910
Epoch 5/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8544 - mse: 0.8543 - val_loss: 0.8830 - val_mse: 0.8828


Epoch 6/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8501 - mse: 0.8499 - val_loss: 0.8799 - val_mse: 0.8797


...

Epoch 2/30
250/250 [==============================] - 3s 12ms/step - loss: 0.8832 - mse: 0.8831 - val_loss: 0.8872 - val_mse: 0.8871


Epoch 3/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8671 - mse: 0.8670 - val_loss: 0.8914 - val_mse: 0.8912
Epoch 4/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8565 - mse: 0.8564 - val_loss: 0.8710 - val_mse: 0.8708


Epoch 5/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8376 - mse: 0.8374 - val_loss: 0.8666 - val_mse: 0.8664


Epoch 6/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8212 - mse: 0.8210 - val_loss: 0.8567 - val_mse: 0.8565


Epoch 7/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8053 - mse: 0.8050 - val_loss: 0.8599 - val_mse: 0.8596
Epoch 8/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7896 - mse: 0.7893 - val_loss: 0.8596 - val_mse: 0.8594
Epoch 9/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7781 - mse: 0.7778 - val_loss: 0.8757 - val_mse: 0.8754
Epoch 10/30
250/250 [==============================] - 3s 12ms/step - loss: 0.7674 - mse: 0.7670 - val_loss: 0.8839 - val_mse: 0.8835
Epoch 11/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7488 - mse: 0.7485 - val_loss: 0.8647 - val_mse: 0.8643
test MSE 0.8771


In [28]:
# OPNN
dict_OPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_outter': True}
ls_result = calc_nn_based_model(df_movie_all, 
                                PNN, dict_OPNN,
                                feature_names, 
                                'movie_lens', 'OPNN')
all_result.extend(ls_result)

Epoch 1/30
250/250 [==============================] - 7s 15ms/step - loss: 1.9266 - mse: 1.9265 - val_loss: 0.9025 - val_mse: 0.9024


Epoch 2/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8807 - mse: 0.8807 - val_loss: 0.8941 - val_mse: 0.8940


Epoch 3/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8672 - mse: 0.8671 - val_loss: 0.8825 - val_mse: 0.8824


Epoch 4/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8584 - mse: 0.8582 - val_loss: 0.8784 - val_mse: 0.8782


Epoch 5/30
250/250 [==============================] - 4s 14ms/step - loss: 0.8423 - mse: 0.8421 - val_loss: 0.8648 - val_mse: 0.8646


...

Epoch 3/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8565 - mse: 0.8564 - val_loss: 0.8711 - val_mse: 0.8710


Epoch 4/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8326 - mse: 0.8324 - val_loss: 0.8725 - val_mse: 0.8724
Epoch 5/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8188 - mse: 0.8186 - val_loss: 0.8860 - val_mse: 0.8858
Epoch 6/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8010 - mse: 0.8008 - val_loss: 0.8836 - val_mse: 0.8833
Epoch 7/30
250/250 [==============================] - 4s 15ms/step - loss: 0.7878 - mse: 0.7875 - val_loss: 0.8749 - val_mse: 0.8746
Epoch 8/30
250/250 [==============================] - 4s 15ms/step - loss: 0.7692 - mse: 0.7689 - val_loss: 0.8823 - val_mse: 0.8821
test MSE 0.8784
Epoch 1/30
250/250 [==============================] - 7s 14ms/step - loss: 1.9007 - mse: 1.9006 - val_loss: 0.8983 - val_mse: 0.8983


Epoch 2/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8820 - mse: 0.8819 - val_loss: 0.9015 - val_mse: 0.9013
Epoch 3/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8605 - mse: 0.8604 - val_loss: 0.8685 - val_mse: 0.8683


Epoch 4/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8350 - mse: 0.8348 - val_loss: 0.8610 - val_mse: 0.8608


Epoch 5/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8216 - mse: 0.8214 - val_loss: 0.8689 - val_mse: 0.8687
Epoch 6/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8120 - mse: 0.8117 - val_loss: 0.8661 - val_mse: 0.8659
Epoch 7/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8057 - mse: 0.8055 - val_loss: 0.8684 - val_mse: 0.8681
Epoch 8/30
250/250 [==============================] - 3s 14ms/step - loss: 0.7988 - mse: 0.7985 - val_loss: 0.8690 - val_mse: 0.8687
Epoch 9/30
250/250 [==============================] - 3s 14ms/step - loss: 0.7925 - mse: 0.7922 - val_loss: 0.8697 - val_mse: 0.8694
test MSE 0.8832


In [29]:
# PIN

In [30]:
# CCPM
dict_CCPM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                CCPM, dict_CCPM,
                                feature_names, 
                                'movie_lens', 'CCPM')
all_result.extend(ls_result)

Epoch 1/30
250/250 [==============================] - 10s 19ms/step - loss: 2.0023 - mse: 2.0023 - val_loss: 1.1055 - val_mse: 1.1054


Epoch 2/30
250/250 [==============================] - 4s 16ms/step - loss: 0.9952 - mse: 0.9950 - val_loss: 0.9210 - val_mse: 0.9207


Epoch 3/30
250/250 [==============================] - 4s 16ms/step - loss: 0.8785 - mse: 0.8781 - val_loss: 0.8972 - val_mse: 0.8967


Epoch 4/30
250/250 [==============================] - 4s 17ms/step - loss: 0.8495 - mse: 0.8490 - val_loss: 0.8827 - val_mse: 0.8822


Epoch 5/30
250/250 [==============================] - 4s 17ms/step - loss: 0.8327 - mse: 0.8321 - val_loss: 0.8808 - val_mse: 0.8801


...

Epoch 3/30
250/250 [==============================] - 4s 16ms/step - loss: 0.8917 - mse: 0.8913 - val_loss: 0.8864 - val_mse: 0.8859


Epoch 4/30
250/250 [==============================] - 4s 16ms/step - loss: 0.8518 - mse: 0.8513 - val_loss: 0.8853 - val_mse: 0.8847


Epoch 5/30
250/250 [==============================] - 4s 16ms/step - loss: 0.8344 - mse: 0.8338 - val_loss: 0.8783 - val_mse: 0.8776


Epoch 6/30
250/250 [==============================] - 4s 17ms/step - loss: 0.8180 - mse: 0.8173 - val_loss: 0.8722 - val_mse: 0.8715


Epoch 7/30
250/250 [==============================] - 4s 17ms/step - loss: 0.8095 - mse: 0.8087 - val_loss: 0.8727 - val_mse: 0.8719
Epoch 8/30
250/250 [==============================] - 4s 17ms/step - loss: 0.7987 - mse: 0.7979 - val_loss: 0.8754 - val_mse: 0.8745
Epoch 9/30
250/250 [==============================] - 4s 16ms/step - loss: 0.7903 - mse: 0.7894 - val_loss: 0.8761 - val_mse: 0.8751
Epoch 10/30
250/250 [==============================] - 4s 17ms/step - loss: 0.7780 - mse: 0.7771 - val_loss: 0.8758 - val_mse: 0.8748
Epoch 11/30
250/250 [==============================] - 4s 17ms/step - loss: 0.7702 - mse: 0.7692 - val_loss: 0.8883 - val_mse: 0.8872
test MSE 0.8933


In [31]:
# WD
dict_WD = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                WDL, dict_WD,
                                feature_names, 
                                'movie_lens', 'WD')
all_result.extend(ls_result)

Epoch 1/30
250/250 [==============================] - 8s 15ms/step - loss: 1.7844 - mse: 1.7844 - val_loss: 0.8996 - val_mse: 0.8996


Epoch 2/30
250/250 [==============================] - 3s 12ms/step - loss: 0.8804 - mse: 0.8803 - val_loss: 0.8847 - val_mse: 0.8845


Epoch 3/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8652 - mse: 0.8650 - val_loss: 0.9065 - val_mse: 0.9063
Epoch 4/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8608 - mse: 0.8606 - val_loss: 0.8833 - val_mse: 0.8831


Epoch 5/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8559 - mse: 0.8557 - val_loss: 0.8791 - val_mse: 0.8789


Epoch 6/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8524 - mse: 0.8521 - val_loss: 0.8821 - val_mse: 0.8818
Epoch 7/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8479 - mse: 0.8477 - val_loss: 0.8752 - val_mse: 0.8749


...

Epoch 8/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8371 - mse: 0.8368 - val_loss: 0.8673 - val_mse: 0.8670


Epoch 9/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8241 - mse: 0.8237 - val_loss: 0.8582 - val_mse: 0.8579


Epoch 10/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8161 - mse: 0.8157 - val_loss: 0.8558 - val_mse: 0.8554


Epoch 11/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8058 - mse: 0.8054 - val_loss: 0.8568 - val_mse: 0.8564
Epoch 12/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7983 - mse: 0.7978 - val_loss: 0.8547 - val_mse: 0.8542


Epoch 13/30
250/250 [==============================] - 3s 14ms/step - loss: 0.7880 - mse: 0.7875 - val_loss: 0.8601 - val_mse: 0.8596
Epoch 14/30
250/250 [==============================] - 3s 14ms/step - loss: 0.7823 - mse: 0.7817 - val_loss: 0.8579 - val_mse: 0.8573
Epoch 15/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7767 - mse: 0.7761 - val_loss: 0.8799 - val_mse: 0.8793
Epoch 16/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7695 - mse: 0.7689 - val_loss: 0.8585 - val_mse: 0.8579
Epoch 17/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7622 - mse: 0.7615 - val_loss: 0.8645 - val_mse: 0.8638
test MSE 0.8799


In [32]:
# DC
dict_DC = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                DCN, dict_DC,
                                feature_names, 
                                'movie_lens', 'DC')
all_result.extend(ls_result)

CrossNet parameterization: vector
Epoch 1/30
250/250 [==============================] - 7s 15ms/step - loss: 1.7447 - mse: 1.7446 - val_loss: 0.9086 - val_mse: 0.9085


Epoch 2/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8820 - mse: 0.8818 - val_loss: 0.9011 - val_mse: 0.9010


Epoch 3/30
250/250 [==============================] - 4s 14ms/step - loss: 0.8661 - mse: 0.8659 - val_loss: 0.8833 - val_mse: 0.8831


Epoch 4/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8582 - mse: 0.8580 - val_loss: 0.8835 - val_mse: 0.8833
Epoch 5/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8565 - mse: 0.8562 - val_loss: 0.8914 - val_mse: 0.8911
Epoch 6/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8518 - mse: 0.8515 - val_loss: 0.8917 - val_mse: 0.8914
Epoch 7/30
250/250 [==============================] - 4s 14ms/step - loss: 0.8464 - mse: 0.8461 - val_loss: 0.8787 - val_mse: 0.8784


Epoch 8/30
250/250 [==============================] - 4s 14ms/step - loss: 0.8411 - mse: 0.8408 - val_loss: 0.8712 - val_mse: 0.8709


...

Epoch 7/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8391 - mse: 0.8388 - val_loss: 0.8667 - val_mse: 0.8664


Epoch 8/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8268 - mse: 0.8265 - val_loss: 0.8602 - val_mse: 0.8599


Epoch 9/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8190 - mse: 0.8187 - val_loss: 0.8620 - val_mse: 0.8616
Epoch 10/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8097 - mse: 0.8092 - val_loss: 0.8736 - val_mse: 0.8732
Epoch 11/30
250/250 [==============================] - 4s 14ms/step - loss: 0.8025 - mse: 0.8020 - val_loss: 0.8574 - val_mse: 0.8569


Epoch 12/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7944 - mse: 0.7939 - val_loss: 0.8557 - val_mse: 0.8552


Epoch 13/30
250/250 [==============================] - 3s 14ms/step - loss: 0.7856 - mse: 0.7850 - val_loss: 0.8569 - val_mse: 0.8563
Epoch 14/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7779 - mse: 0.7773 - val_loss: 0.8574 - val_mse: 0.8568
Epoch 15/30
250/250 [==============================] - 4s 15ms/step - loss: 0.7711 - mse: 0.7705 - val_loss: 0.8577 - val_mse: 0.8570
Epoch 16/30
250/250 [==============================] - 3s 14ms/step - loss: 0.7653 - mse: 0.7646 - val_loss: 0.8611 - val_mse: 0.8604
Epoch 17/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7531 - mse: 0.7524 - val_loss: 0.8691 - val_mse: 0.8683
test MSE 0.8775


In [33]:
# NFM
dict_NFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                NFM, dict_NFM,
                                feature_names, 
                                'movie_lens', 'NFM')
all_result.extend(ls_result)

Epoch 1/30
250/250 [==============================] - 8s 15ms/step - loss: 1.9396 - mse: 1.9395 - val_loss: 0.9482 - val_mse: 0.9480


Epoch 2/30
250/250 [==============================] - 3s 13ms/step - loss: 0.9011 - mse: 0.9009 - val_loss: 0.8973 - val_mse: 0.8971


Epoch 3/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8724 - mse: 0.8721 - val_loss: 0.8998 - val_mse: 0.8995
Epoch 4/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8659 - mse: 0.8655 - val_loss: 0.8841 - val_mse: 0.8838


Epoch 5/30
250/250 [==============================] - 4s 14ms/step - loss: 0.8587 - mse: 0.8584 - val_loss: 0.8893 - val_mse: 0.8889
Epoch 6/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8551 - mse: 0.8547 - val_loss: 0.8914 - val_mse: 0.8910
Epoch 7/30
250/250 [==============================] - 4s 14ms/step - loss: 0.8473 - mse: 0.8469 - val_loss: 0.8753 - val_mse: 0.8748


Epoch 8/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8368 - mse: 0.8362 - val_loss: 0.8787 - val_mse: 0.8782
Epoch 9/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8293 - mse: 0.8287 - val_loss: 0.8823 - val_mse: 0.8817
Epoch 10/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8213 - mse: 0.8207 - val_loss: 0.8698 - val_mse: 0.8692


...

Epoch 2/30
250/250 [==============================] - 3s 13ms/step - loss: 0.9002 - mse: 0.9000 - val_loss: 0.8883 - val_mse: 0.8881


Epoch 3/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8697 - mse: 0.8694 - val_loss: 0.8898 - val_mse: 0.8895
Epoch 4/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8615 - mse: 0.8611 - val_loss: 0.8810 - val_mse: 0.8807


Epoch 5/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8590 - mse: 0.8586 - val_loss: 0.8816 - val_mse: 0.8812
Epoch 6/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8532 - mse: 0.8528 - val_loss: 0.8875 - val_mse: 0.8870
Epoch 7/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8474 - mse: 0.8469 - val_loss: 0.9053 - val_mse: 0.9048
Epoch 8/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8379 - mse: 0.8373 - val_loss: 0.8882 - val_mse: 0.8876
Epoch 9/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8298 - mse: 0.8292 - val_loss: 0.8653 - val_mse: 0.8647


Epoch 10/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8187 - mse: 0.8181 - val_loss: 0.8729 - val_mse: 0.8722
Epoch 11/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8112 - mse: 0.8105 - val_loss: 0.8654 - val_mse: 0.8647


Epoch 12/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8044 - mse: 0.8036 - val_loss: 0.8664 - val_mse: 0.8656
Epoch 13/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7979 - mse: 0.7971 - val_loss: 0.8722 - val_mse: 0.8713
Epoch 14/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7948 - mse: 0.7939 - val_loss: 0.8665 - val_mse: 0.8656
Epoch 15/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7905 - mse: 0.7896 - val_loss: 0.8791 - val_mse: 0.8782
Epoch 16/30
250/250 [==============================] - 3s 13ms/step - loss: 0.7848 - mse: 0.7838 - val_loss: 0.8685 - val_mse: 0.8675
test MSE 0.8854


In [34]:
# deepFM
dict_deepFM = {'linear_feature_columns': linear_feature_columns, 
               'dnn_feature_columns': dnn_feature_columns,
               'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                DeepFM, dict_deepFM,
                                feature_names, 
                                'movie_lens', 'deepFM')
all_result.extend(ls_result)

Epoch 1/30
250/250 [==============================] - 7s 14ms/step - loss: 1.7596 - mse: 1.7595 - val_loss: 0.9063 - val_mse: 0.9063


Epoch 2/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8816 - mse: 0.8815 - val_loss: 0.8838 - val_mse: 0.8837


Epoch 3/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8660 - mse: 0.8658 - val_loss: 0.8817 - val_mse: 0.8816


Epoch 4/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8598 - mse: 0.8596 - val_loss: 0.8872 - val_mse: 0.8870
Epoch 5/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8562 - mse: 0.8560 - val_loss: 0.8780 - val_mse: 0.8778


Epoch 6/30
250/250 [==============================] - 3s 14ms/step - loss: 0.8487 - mse: 0.8485 - val_loss: 0.8757 - val_mse: 0.8755


...

Epoch 6/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8538 - mse: 0.8536 - val_loss: 0.8747 - val_mse: 0.8744


Epoch 7/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8466 - mse: 0.8464 - val_loss: 0.8714 - val_mse: 0.8712


Epoch 8/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8324 - mse: 0.8321 - val_loss: 0.8749 - val_mse: 0.8746
Epoch 9/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8243 - mse: 0.8239 - val_loss: 0.8583 - val_mse: 0.8580


Epoch 10/30
250/250 [==============================] - 3s 13ms/step - loss: 0.8129 - mse: 0.8125 - val_loss: 0.8617 - val_mse: 0.8613
Epoch 11/30
250/250 [==============================] - 4s 14ms/step - loss: 0.8059 - mse: 0.8055 - val_loss: 0.8700 - val_mse: 0.8696
Epoch 12/30
250/250 [==============================] - 4s 15ms/step - loss: 0.7973 - mse: 0.7969 - val_loss: 0.8706 - val_mse: 0.8701
Epoch 13/30
250/250 [==============================] - 4s 16ms/step - loss: 0.7872 - mse: 0.7868 - val_loss: 0.8612 - val_mse: 0.8607
Epoch 14/30
250/250 [==============================] - 4s 15ms/step - loss: 0.7780 - mse: 0.7775 - val_loss: 0.8579 - val_mse: 0.8574


Epoch 15/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7700 - mse: 0.7694 - val_loss: 0.8621 - val_mse: 0.8615
Epoch 16/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7611 - mse: 0.7605 - val_loss: 0.8643 - val_mse: 0.8637
Epoch 17/30
250/250 [==============================] - 4s 15ms/step - loss: 0.7517 - mse: 0.7510 - val_loss: 0.8704 - val_mse: 0.8697
Epoch 18/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7422 - mse: 0.7416 - val_loss: 0.8698 - val_mse: 0.8692
Epoch 19/30
250/250 [==============================] - 4s 15ms/step - loss: 0.7281 - mse: 0.7274 - val_loss: 0.8684 - val_mse: 0.8677
test MSE 0.8795


In [35]:
# AFM
dict_AFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                AFM, dict_AFM,
                                feature_names, 
                                'movie_lens', 'AFM')
all_result.extend(ls_result)

Epoch 1/30
250/250 [==============================] - 9s 15ms/step - loss: 4.3529 - mse: 4.3526 - val_loss: 1.2306 - val_mse: 1.2299


Epoch 2/30
250/250 [==============================] - 4s 14ms/step - loss: 1.1753 - mse: 1.1743 - val_loss: 1.1321 - val_mse: 1.1309


Epoch 3/30
250/250 [==============================] - 4s 15ms/step - loss: 1.0936 - mse: 1.0919 - val_loss: 1.0651 - val_mse: 1.0630


Epoch 4/30
250/250 [==============================] - 4s 15ms/step - loss: 0.9942 - mse: 0.9915 - val_loss: 0.9584 - val_mse: 0.9551


Epoch 5/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8973 - mse: 0.8937 - val_loss: 0.9143 - val_mse: 0.9105


...

Epoch 19/30
250/250 [==============================] - 4s 15ms/step - loss: 0.8006 - mse: 0.7950 - val_loss: 0.8799 - val_mse: 0.8742


Epoch 20/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7982 - mse: 0.7925 - val_loss: 0.8795 - val_mse: 0.8737


Epoch 21/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7965 - mse: 0.7907 - val_loss: 0.8789 - val_mse: 0.8731


Epoch 22/30
250/250 [==============================] - 3s 14ms/step - loss: 0.7938 - mse: 0.7879 - val_loss: 0.8804 - val_mse: 0.8745
Epoch 23/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7919 - mse: 0.7859 - val_loss: 0.8790 - val_mse: 0.8730


Epoch 24/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7898 - mse: 0.7838 - val_loss: 0.8812 - val_mse: 0.8751
Epoch 25/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7884 - mse: 0.7823 - val_loss: 0.8797 - val_mse: 0.8735
Epoch 26/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7864 - mse: 0.7802 - val_loss: 0.8796 - val_mse: 0.8733
Epoch 27/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7845 - mse: 0.7782 - val_loss: 0.8802 - val_mse: 0.8739
Epoch 28/30
250/250 [==============================] - 4s 14ms/step - loss: 0.7828 - mse: 0.7764 - val_loss: 0.8812 - val_mse: 0.8749
test MSE 0.8915


In [36]:
# xDeepFM
dict_xDeepFM = {'linear_feature_columns': linear_feature_columns,
                'dnn_feature_columns': dnn_feature_columns,
                'task': 'regression'}
ls_result = calc_nn_based_model(df_movie_all, 
                                xDeepFM, dict_xDeepFM,
                                feature_names, 
                                'movie_lens', 'xDeepFM')
all_result.extend(ls_result)

Epoch 1/30
250/250 [==============================] - 8s 18ms/step - loss: 1.6724 - mse: 1.6723 - val_loss: 0.9034 - val_mse: 0.9033


Epoch 2/30
250/250 [==============================] - 4s 16ms/step - loss: 0.8787 - mse: 0.8786 - val_loss: 0.8841 - val_mse: 0.8840


Epoch 3/30
250/250 [==============================] - 4s 16ms/step - loss: 0.8640 - mse: 0.8639 - val_loss: 0.8809 - val_mse: 0.8808


Epoch 4/30
250/250 [==============================] - 4s 16ms/step - loss: 0.8587 - mse: 0.8585 - val_loss: 0.8876 - val_mse: 0.8874
Epoch 5/30
250/250 [==============================] - 4s 16ms/step - loss: 0.8487 - mse: 0.8485 - val_loss: 0.8724 - val_mse: 0.8722


Epoch 6/30
250/250 [==============================] - 4s 16ms/step - loss: 0.8350 - mse: 0.8348 - val_loss: 0.8686 - val_mse: 0.8683


...

Epoch 7/30
250/250 [==============================] - 4s 17ms/step - loss: 0.8216 - mse: 0.8213 - val_loss: 0.8579 - val_mse: 0.8576


Epoch 8/30
250/250 [==============================] - 4s 17ms/step - loss: 0.8113 - mse: 0.8110 - val_loss: 0.8667 - val_mse: 0.8664
Epoch 9/30
250/250 [==============================] - 4s 17ms/step - loss: 0.8068 - mse: 0.8065 - val_loss: 0.8564 - val_mse: 0.8561


Epoch 10/30
250/250 [==============================] - 4s 18ms/step - loss: 0.8014 - mse: 0.8010 - val_loss: 0.8555 - val_mse: 0.8551


Epoch 11/30
250/250 [==============================] - 4s 17ms/step - loss: 0.7888 - mse: 0.7884 - val_loss: 0.8529 - val_mse: 0.8524


Epoch 12/30
250/250 [==============================] - 4s 17ms/step - loss: 0.7791 - mse: 0.7786 - val_loss: 0.8560 - val_mse: 0.8556
Epoch 13/30
250/250 [==============================] - 4s 17ms/step - loss: 0.7687 - mse: 0.7682 - val_loss: 0.8543 - val_mse: 0.8538
Epoch 14/30
250/250 [==============================] - 4s 17ms/step - loss: 0.7540 - mse: 0.7535 - val_loss: 0.8568 - val_mse: 0.8563
Epoch 15/30
250/250 [==============================] - 4s 16ms/step - loss: 0.7421 - mse: 0.7415 - val_loss: 0.8623 - val_mse: 0.8617
Epoch 16/30
250/250 [==============================] - 4s 17ms/step - loss: 0.7303 - mse: 0.7296 - val_loss: 0.8608 - val_mse: 0.8601
test MSE 0.8722


### yelp

In [37]:
df_yelp_all = pd.merge(df_yelp_inter, df_yelp_user_feats, on='user_id', how='left')
df_yelp_all = pd.merge(df_yelp_all, df_movie_item_feats, on='item_id', how='left')
df_yelp_all['rating'] = df_yelp_all.pop('rating')
df_yelp_all.iloc[:, :-1] = df_yelp_all.iloc[:, :-1].fillna('0')
print(df_yelp_all.shape)
df_yelp_all.head(3)

(188456, 32)


,user_id,item_id,compliment1,compliment10,compliment11,compliment2,compliment3,compliment4,compliment5,compliment6,...,genre18,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,rating
0,2,186,1,0,1,1,1,1,1,1,...,0,0,0,0,1,0,0,0,0,5
1,2,205,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,5
2,2,209,1,0,1,1,1,1,1,1,...,0,0,0,0,1,0,0,1,0,4


In [38]:
linear_feature_columns, dnn_feature_columns, feature_names = get_feature_columns(df_yelp_all)
df_yelp_all.head(3)

,user_id,item_id,compliment1,compliment10,compliment11,compliment2,compliment3,compliment4,compliment5,compliment6,...,genre18,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,rating
0,3867,5209,1,0,1,1,1,1,1,1,...,0,0,0,0,1,0,0,0,0,5
1,3867,5418,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,5
2,3867,5461,1,0,1,1,1,1,1,1,...,0,0,0,0,1,0,0,1,0,4


In [39]:
# FNN
dict_FNN = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                FNN, dict_FNN,
                                feature_names, 
                                'yelp', 'FNN')
all_result.extend(ls_result)

Epoch 1/30
472/472 [==============================] - 12s 17ms/step - loss: 1.5333 - mse: 1.5331 - val_loss: 1.1024 - val_mse: 1.1019


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9658 - mse: 0.9651 - val_loss: 1.0960 - val_mse: 1.0950


Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9099 - mse: 0.9086 - val_loss: 1.1407 - val_mse: 1.1392
Epoch 4/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8826 - mse: 0.8808 - val_loss: 1.1014 - val_mse: 1.0994
Epoch 5/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8627 - mse: 0.8604 - val_loss: 1.1524 - val_mse: 1.1498
Epoch 6/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8444 - mse: 0.8416 - val_loss: 1.1358 - val_mse: 1.1327
Epoch 7/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8210 - mse: 0.8177 - val_loss: 1.1510 - val_mse: 1.1474
test MSE 1.0727
Epoch 1/30
472/472 [==============================] - 14s 19ms/step - loss: 1.6225 - mse: 1.6222 - val_loss: 1.1096 - val_mse: 1.1091


Epoch 2/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9679 - mse: 0.9671 - val_loss: 1.1014 - val_mse: 1.1003


Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9088 - mse: 0.9074 - val_loss: 1.1069 - val_mse: 1.1053
Epoch 4/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8803 - mse: 0.8784 - val_loss: 1.1151 - val_mse: 1.1129
Epoch 5/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8626 - mse: 0.8602 - val_loss: 1.1191 - val_mse: 1.1164
Epoch 6/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8498 - mse: 0.8468 - val_loss: 1.1421 - val_mse: 1.1390
Epoch 7/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8330 - mse: 0.8295 - val_loss: 1.1431 - val_mse: 1.1393
test MSE 1.07
Epoch 1/30
472/472 [==============================] - 12s 17ms/step - loss: 1.6193 - mse: 1.6190 - val_loss: 1.1107 - val_mse: 1.1102


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9634 - mse: 0.9626 - val_loss: 1.1151 - val_mse: 1.1140
Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9039 - mse: 0.9026 - val_loss: 1.1291 - val_mse: 1.1275
Epoch 4/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8758 - mse: 0.8740 - val_loss: 1.1178 - val_mse: 1.1157
Epoch 5/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8601 - mse: 0.8577 - val_loss: 1.1529 - val_mse: 1.1502
Epoch 6/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8430 - mse: 0.8401 - val_loss: 1.1487 - val_mse: 1.1456
test MSE 1.0817
Epoch 1/30
472/472 [==============================] - 15s 19ms/step - loss: 1.5472 - mse: 1.5470 - val_loss: 1.1220 - val_mse: 1.1215


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9679 - mse: 0.9671 - val_loss: 1.0931 - val_mse: 1.0920


Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9104 - mse: 0.9092 - val_loss: 1.0937 - val_mse: 1.0922
Epoch 4/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8841 - mse: 0.8823 - val_loss: 1.1072 - val_mse: 1.1052
Epoch 5/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8646 - mse: 0.8623 - val_loss: 1.1068 - val_mse: 1.1042
Epoch 6/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8539 - mse: 0.8511 - val_loss: 1.1240 - val_mse: 1.1209
Epoch 7/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8337 - mse: 0.8304 - val_loss: 1.1441 - val_mse: 1.1405
test MSE 1.0672
Epoch 1/30
472/472 [==============================] - 14s 17ms/step - loss: 1.5577 - mse: 1.5575 - val_loss: 1.1006 - val_mse: 1.1001


Epoch 2/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9677 - mse: 0.9669 - val_loss: 1.0970 - val_mse: 1.0960


Epoch 3/30
472/472 [==============================] - 9s 18ms/step - loss: 0.9077 - mse: 0.9064 - val_loss: 1.1251 - val_mse: 1.1235
Epoch 4/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8804 - mse: 0.8786 - val_loss: 1.1550 - val_mse: 1.1529
Epoch 5/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8637 - mse: 0.8613 - val_loss: 1.1347 - val_mse: 1.1321
Epoch 6/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8427 - mse: 0.8399 - val_loss: 1.1505 - val_mse: 1.1474
Epoch 7/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8201 - mse: 0.8168 - val_loss: 1.1533 - val_mse: 1.1497
test MSE 1.0844


In [40]:
# IPNN
dict_IPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_inner': True}
ls_result = calc_nn_based_model(df_yelp_all, 
                                PNN, dict_IPNN,
                                feature_names, 
                                'yelp', 'IPNN')
all_result.extend(ls_result)

Epoch 1/30
472/472 [==============================] - 12s 17ms/step - loss: 1.7646 - mse: 1.7644 - val_loss: 1.1127 - val_mse: 1.1123


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9681 - mse: 0.9674 - val_loss: 1.1078 - val_mse: 1.1069


Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9111 - mse: 0.9100 - val_loss: 1.1282 - val_mse: 1.1269
Epoch 4/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8779 - mse: 0.8763 - val_loss: 1.1329 - val_mse: 1.1311
Epoch 5/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8463 - mse: 0.8443 - val_loss: 1.1322 - val_mse: 1.1300
Epoch 6/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8139 - mse: 0.8115 - val_loss: 1.1429 - val_mse: 1.1403
Epoch 7/30
472/472 [==============================] - 8s 18ms/step - loss: 0.7851 - mse: 0.7822 - val_loss: 1.1452 - val_mse: 1.1421
test MSE 1.0715
Epoch 1/30
472/472 [==============================] - 12s 17ms/step - loss: 1.6850 - mse: 1.6848 - val_loss: 1.1077 - val_mse: 1.1073


Epoch 2/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9682 - mse: 0.9676 - val_loss: 1.1037 - val_mse: 1.1028


Epoch 3/30
472/472 [==============================] - 8s 16ms/step - loss: 0.9104 - mse: 0.9093 - val_loss: 1.1033 - val_mse: 1.1020


...

Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9706 - mse: 0.9699 - val_loss: 1.1108 - val_mse: 1.1099


Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9117 - mse: 0.9106 - val_loss: 1.1069 - val_mse: 1.1056


Epoch 4/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8822 - mse: 0.8807 - val_loss: 1.1387 - val_mse: 1.1370
Epoch 5/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8565 - mse: 0.8545 - val_loss: 1.1055 - val_mse: 1.1033


Epoch 6/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8217 - mse: 0.8193 - val_loss: 1.1357 - val_mse: 1.1330
Epoch 7/30
472/472 [==============================] - 8s 18ms/step - loss: 0.7880 - mse: 0.7852 - val_loss: 1.1656 - val_mse: 1.1626
Epoch 8/30
472/472 [==============================] - 8s 17ms/step - loss: 0.7562 - mse: 0.7530 - val_loss: 1.1248 - val_mse: 1.1214
Epoch 9/30
472/472 [==============================] - 8s 17ms/step - loss: 0.7220 - mse: 0.7185 - val_loss: 1.1853 - val_mse: 1.1815
Epoch 10/30
472/472 [==============================] - 8s 17ms/step - loss: 0.6883 - mse: 0.6844 - val_loss: 1.1586 - val_mse: 1.1545
test MSE 1.0957
Epoch 1/30
472/472 [==============================] - 12s 16ms/step - loss: 1.6844 - mse: 1.6842 - val_loss: 1.1025 - val_mse: 1.1020


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9675 - mse: 0.9669 - val_loss: 1.1065 - val_mse: 1.1056
Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9039 - mse: 0.9028 - val_loss: 1.1122 - val_mse: 1.1109
Epoch 4/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8555 - mse: 0.8539 - val_loss: 1.1101 - val_mse: 1.1083
Epoch 5/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8114 - mse: 0.8094 - val_loss: 1.2002 - val_mse: 1.1980
Epoch 6/30
472/472 [==============================] - 8s 17ms/step - loss: 0.7809 - mse: 0.7784 - val_loss: 1.1237 - val_mse: 1.1211
test MSE 1.0803


In [41]:
# OPNN
dict_OPNN = {'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression',
            'use_outter': True}
ls_result = calc_nn_based_model(df_yelp_all, 
                                PNN, dict_OPNN,
                                feature_names, 
                                'yelp', 'OPNN')
all_result.extend(ls_result)

Epoch 1/30
472/472 [==============================] - 13s 20ms/step - loss: 1.6904 - mse: 1.6902 - val_loss: 1.1142 - val_mse: 1.1137


Epoch 2/30
472/472 [==============================] - 10s 21ms/step - loss: 0.9700 - mse: 0.9693 - val_loss: 1.1046 - val_mse: 1.1037


Epoch 3/30
472/472 [==============================] - 10s 21ms/step - loss: 0.9103 - mse: 0.9092 - val_loss: 1.1197 - val_mse: 1.1185
Epoch 4/30
472/472 [==============================] - 10s 21ms/step - loss: 0.8792 - mse: 0.8777 - val_loss: 1.1777 - val_mse: 1.1759
Epoch 5/30
472/472 [==============================] - 10s 21ms/step - loss: 0.8491 - mse: 0.8472 - val_loss: 1.1182 - val_mse: 1.1160
Epoch 6/30
472/472 [==============================] - 10s 21ms/step - loss: 0.8135 - mse: 0.8111 - val_loss: 1.1634 - val_mse: 1.1609
Epoch 7/30
472/472 [==============================] - 10s 21ms/step - loss: 0.7803 - mse: 0.7776 - val_loss: 1.1455 - val_mse: 1.1425
test MSE 1.0691
Epoch 1/30
472/472 [==============================] - 15s 20ms/step - loss: 1.6508 - mse: 1.6506 - val_loss: 1.1113 - val_mse: 1.1109


Epoch 2/30
472/472 [==============================] - 10s 20ms/step - loss: 0.9694 - mse: 0.9688 - val_loss: 1.1035 - val_mse: 1.1027


Epoch 3/30
472/472 [==============================] - 10s 20ms/step - loss: 0.9072 - mse: 0.9062 - val_loss: 1.1097 - val_mse: 1.1084
Epoch 4/30
472/472 [==============================] - 10s 21ms/step - loss: 0.8737 - mse: 0.8722 - val_loss: 1.1618 - val_mse: 1.1600
Epoch 5/30
472/472 [==============================] - 10s 20ms/step - loss: 0.8370 - mse: 0.8351 - val_loss: 1.1283 - val_mse: 1.1262
Epoch 6/30
472/472 [==============================] - 10s 21ms/step - loss: 0.7989 - mse: 0.7966 - val_loss: 1.1285 - val_mse: 1.1260
Epoch 7/30
472/472 [==============================] - 10s 20ms/step - loss: 0.7665 - mse: 0.7638 - val_loss: 1.1504 - val_mse: 1.1475
test MSE 1.0594
Epoch 1/30
472/472 [==============================] - 16s 21ms/step - loss: 1.6565 - mse: 1.6563 - val_loss: 1.1262 - val_mse: 1.1258


Epoch 2/30
472/472 [==============================] - 9s 20ms/step - loss: 0.9635 - mse: 0.9629 - val_loss: 1.1222 - val_mse: 1.1213


Epoch 3/30
472/472 [==============================] - 10s 20ms/step - loss: 0.8975 - mse: 0.8964 - val_loss: 1.1152 - val_mse: 1.1139


Epoch 4/30
472/472 [==============================] - 9s 20ms/step - loss: 0.8451 - mse: 0.8436 - val_loss: 1.1649 - val_mse: 1.1632
Epoch 5/30
472/472 [==============================] - 10s 20ms/step - loss: 0.8019 - mse: 0.8000 - val_loss: 1.1751 - val_mse: 1.1729
Epoch 6/30
472/472 [==============================] - 10s 20ms/step - loss: 0.7699 - mse: 0.7676 - val_loss: 1.1585 - val_mse: 1.1559
Epoch 7/30
472/472 [==============================] - 10s 21ms/step - loss: 0.7434 - mse: 0.7407 - val_loss: 1.1590 - val_mse: 1.1561
Epoch 8/30
472/472 [==============================] - 10s 20ms/step - loss: 0.7073 - mse: 0.7042 - val_loss: 1.1406 - val_mse: 1.1373
test MSE 1.1116
Epoch 1/30
472/472 [==============================] - 17s 22ms/step - loss: 1.6373 - mse: 1.6371 - val_loss: 1.1008 - val_mse: 1.1004


Epoch 2/30
472/472 [==============================] - 10s 21ms/step - loss: 0.9696 - mse: 0.9689 - val_loss: 1.1100 - val_mse: 1.1092
Epoch 3/30
472/472 [==============================] - 10s 21ms/step - loss: 0.9112 - mse: 0.9101 - val_loss: 1.1717 - val_mse: 1.1705
Epoch 4/30
472/472 [==============================] - 10s 21ms/step - loss: 0.8738 - mse: 0.8723 - val_loss: 1.1060 - val_mse: 1.1043
Epoch 5/30
472/472 [==============================] - 10s 21ms/step - loss: 0.8334 - mse: 0.8316 - val_loss: 1.1041 - val_mse: 1.1020
Epoch 6/30
472/472 [==============================] - 10s 21ms/step - loss: 0.7967 - mse: 0.7944 - val_loss: 1.1492 - val_mse: 1.1468
test MSE 1.0742
Epoch 1/30
472/472 [==============================] - 13s 20ms/step - loss: 1.6390 - mse: 1.6388 - val_loss: 1.1009 - val_mse: 1.1004


Epoch 2/30
472/472 [==============================] - 10s 22ms/step - loss: 0.9672 - mse: 0.9666 - val_loss: 1.1505 - val_mse: 1.1496
Epoch 3/30
472/472 [==============================] - 10s 21ms/step - loss: 0.9098 - mse: 0.9087 - val_loss: 1.1289 - val_mse: 1.1276
Epoch 4/30
472/472 [==============================] - 10s 20ms/step - loss: 0.8780 - mse: 0.8765 - val_loss: 1.1342 - val_mse: 1.1325
Epoch 5/30
472/472 [==============================] - 10s 21ms/step - loss: 0.8427 - mse: 0.8408 - val_loss: 1.1690 - val_mse: 1.1668
Epoch 6/30
472/472 [==============================] - 10s 21ms/step - loss: 0.8045 - mse: 0.8022 - val_loss: 1.1454 - val_mse: 1.1429
test MSE 1.0828


In [42]:
# PIN

In [43]:
# CCPM
dict_CCPM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                CCPM, dict_CCPM,
                                feature_names, 
                                'yelp', 'CCPM')
all_result.extend(ls_result)

Epoch 1/30
472/472 [==============================] - 16s 23ms/step - loss: 1.7012 - mse: 1.7010 - val_loss: 1.1194 - val_mse: 1.1187


Epoch 2/30
472/472 [==============================] - 11s 22ms/step - loss: 0.9995 - mse: 0.9981 - val_loss: 1.0954 - val_mse: 1.0936


Epoch 3/30
472/472 [==============================] - 11s 22ms/step - loss: 0.9114 - mse: 0.9092 - val_loss: 1.1030 - val_mse: 1.1004
Epoch 4/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8735 - mse: 0.8706 - val_loss: 1.1792 - val_mse: 1.1759
Epoch 5/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8500 - mse: 0.8463 - val_loss: 1.1254 - val_mse: 1.1214
Epoch 6/30
472/472 [==============================] - 11s 22ms/step - loss: 0.8234 - mse: 0.8191 - val_loss: 1.1383 - val_mse: 1.1336
Epoch 7/30
472/472 [==============================] - 10s 22ms/step - loss: 0.7899 - mse: 0.7850 - val_loss: 1.1762 - val_mse: 1.1710
test MSE 1.0688
Epoch 1/30
472/472 [==============================] - 17s 23ms/step - loss: 1.8094 - mse: 1.8092 - val_loss: 1.1707 - val_mse: 1.1701


Epoch 2/30
472/472 [==============================] - 10s 22ms/step - loss: 1.0371 - mse: 1.0358 - val_loss: 1.1151 - val_mse: 1.1134


Epoch 3/30
472/472 [==============================] - 10s 22ms/step - loss: 0.9248 - mse: 0.9227 - val_loss: 1.1084 - val_mse: 1.1059


...

Epoch 4/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8652 - mse: 0.8623 - val_loss: 1.1287 - val_mse: 1.1254
Epoch 5/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8031 - mse: 0.7994 - val_loss: 1.1849 - val_mse: 1.1809
Epoch 6/30
472/472 [==============================] - 11s 22ms/step - loss: 0.7421 - mse: 0.7378 - val_loss: 1.2327 - val_mse: 1.2279
Epoch 7/30
472/472 [==============================] - 11s 23ms/step - loss: 0.6884 - mse: 0.6834 - val_loss: 1.2470 - val_mse: 1.2417
Epoch 8/30
472/472 [==============================] - 10s 22ms/step - loss: 0.6374 - mse: 0.6317 - val_loss: 1.2854 - val_mse: 1.2795
test MSE 1.075
Epoch 1/30
472/472 [==============================] - 15s 22ms/step - loss: 1.7978 - mse: 1.7977 - val_loss: 1.1854 - val_mse: 1.1849


Epoch 2/30
472/472 [==============================] - 11s 23ms/step - loss: 1.1184 - mse: 1.1172 - val_loss: 1.0994 - val_mse: 1.0973


Epoch 3/30
472/472 [==============================] - 11s 22ms/step - loss: 0.9613 - mse: 0.9585 - val_loss: 1.1859 - val_mse: 1.1826
Epoch 4/30
472/472 [==============================] - 10s 22ms/step - loss: 0.8809 - mse: 0.8771 - val_loss: 1.1363 - val_mse: 1.1320
Epoch 5/30
472/472 [==============================] - 11s 22ms/step - loss: 0.8324 - mse: 0.8278 - val_loss: 1.1720 - val_mse: 1.1670
Epoch 6/30
472/472 [==============================] - 11s 22ms/step - loss: 0.7860 - mse: 0.7806 - val_loss: 1.3326 - val_mse: 1.3269
Epoch 7/30
472/472 [==============================] - 11s 22ms/step - loss: 0.7456 - mse: 0.7395 - val_loss: 1.2437 - val_mse: 1.2373
test MSE 1.0837
Epoch 1/30
472/472 [==============================] - 15s 21ms/step - loss: 1.7894 - mse: 1.7892 - val_loss: 1.1752 - val_mse: 1.1747


Epoch 2/30
472/472 [==============================] - 11s 23ms/step - loss: 1.0562 - mse: 1.0550 - val_loss: 1.1184 - val_mse: 1.1168


Epoch 3/30
472/472 [==============================] - 11s 23ms/step - loss: 0.9216 - mse: 0.9195 - val_loss: 1.1370 - val_mse: 1.1344
Epoch 4/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8656 - mse: 0.8627 - val_loss: 1.1400 - val_mse: 1.1367
Epoch 5/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8210 - mse: 0.8173 - val_loss: 1.1493 - val_mse: 1.1452
Epoch 6/30
472/472 [==============================] - 11s 22ms/step - loss: 0.7767 - mse: 0.7722 - val_loss: 1.1873 - val_mse: 1.1824
Epoch 7/30
472/472 [==============================] - 11s 23ms/step - loss: 0.7307 - mse: 0.7255 - val_loss: 1.2168 - val_mse: 1.2113
test MSE 1.0794


In [44]:
# WD
dict_WD = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                WDL, dict_WD,
                                feature_names, 
                                'yelp', 'WD')
all_result.extend(ls_result)

Epoch 1/30
472/472 [==============================] - 13s 17ms/step - loss: 1.5819 - mse: 1.5817 - val_loss: 1.1000 - val_mse: 1.0995


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9653 - mse: 0.9645 - val_loss: 1.0938 - val_mse: 1.0928


Epoch 3/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9092 - mse: 0.9078 - val_loss: 1.1008 - val_mse: 1.0992
Epoch 4/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8816 - mse: 0.8797 - val_loss: 1.1106 - val_mse: 1.1085
Epoch 5/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8640 - mse: 0.8616 - val_loss: 1.1265 - val_mse: 1.1238
Epoch 6/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8478 - mse: 0.8449 - val_loss: 1.1223 - val_mse: 1.1192
Epoch 7/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8258 - mse: 0.8224 - val_loss: 1.1268 - val_mse: 1.1231
test MSE 1.0789
Epoch 1/30
472/472 [==============================] - 12s 17ms/step - loss: 1.5815 - mse: 1.5813 - val_loss: 1.1121 - val_mse: 1.1116


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9685 - mse: 0.9677 - val_loss: 1.1456 - val_mse: 1.1446
Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9097 - mse: 0.9084 - val_loss: 1.1128 - val_mse: 1.1113


Epoch 4/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8808 - mse: 0.8789 - val_loss: 1.1128 - val_mse: 1.1107


...

Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9630 - mse: 0.9622 - val_loss: 1.1215 - val_mse: 1.1205
Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9045 - mse: 0.9032 - val_loss: 1.1255 - val_mse: 1.1239
Epoch 4/30
472/472 [==============================] - 8s 16ms/step - loss: 0.8772 - mse: 0.8754 - val_loss: 1.1441 - val_mse: 1.1420
Epoch 5/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8593 - mse: 0.8570 - val_loss: 1.1365 - val_mse: 1.1339
Epoch 6/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8445 - mse: 0.8416 - val_loss: 1.1782 - val_mse: 1.1751
test MSE 1.0803
Epoch 1/30
472/472 [==============================] - 13s 18ms/step - loss: 1.5830 - mse: 1.5828 - val_loss: 1.0944 - val_mse: 1.0939


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9680 - mse: 0.9672 - val_loss: 1.0933 - val_mse: 1.0922


Epoch 3/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9102 - mse: 0.9089 - val_loss: 1.0935 - val_mse: 1.0919


Epoch 4/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8817 - mse: 0.8798 - val_loss: 1.1061 - val_mse: 1.1040
Epoch 5/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8661 - mse: 0.8638 - val_loss: 1.1194 - val_mse: 1.1168
Epoch 6/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8465 - mse: 0.8437 - val_loss: 1.1210 - val_mse: 1.1179
Epoch 7/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8237 - mse: 0.8204 - val_loss: 1.1247 - val_mse: 1.1211
Epoch 8/30
472/472 [==============================] - 9s 19ms/step - loss: 0.7980 - mse: 0.7941 - val_loss: 1.1553 - val_mse: 1.1512
test MSE 1.0726
Epoch 1/30
472/472 [==============================] - 15s 18ms/step - loss: 1.6167 - mse: 1.6165 - val_loss: 1.1035 - val_mse: 1.1030


Epoch 2/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9663 - mse: 0.9655 - val_loss: 1.1215 - val_mse: 1.1204
Epoch 3/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9077 - mse: 0.9063 - val_loss: 1.1390 - val_mse: 1.1374
Epoch 4/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8804 - mse: 0.8785 - val_loss: 1.1562 - val_mse: 1.1541
Epoch 5/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8638 - mse: 0.8614 - val_loss: 1.1378 - val_mse: 1.1351
Epoch 6/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8503 - mse: 0.8474 - val_loss: 1.1441 - val_mse: 1.1409
test MSE 1.0789


In [45]:
# DC
dict_DC = {'linear_feature_columns': linear_feature_columns,
           'dnn_feature_columns': dnn_feature_columns,
           'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                DCN, dict_DC,
                                feature_names, 
                                'yelp', 'DC')
all_result.extend(ls_result)

CrossNet parameterization: vector
Epoch 1/30
472/472 [==============================] - 13s 19ms/step - loss: 1.5172 - mse: 1.5169 - val_loss: 1.1005 - val_mse: 1.0999


Epoch 2/30
472/472 [==============================] - 9s 18ms/step - loss: 0.9669 - mse: 0.9661 - val_loss: 1.1031 - val_mse: 1.1019
Epoch 3/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9065 - mse: 0.9051 - val_loss: 1.1186 - val_mse: 1.1169
Epoch 4/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8801 - mse: 0.8782 - val_loss: 1.1265 - val_mse: 1.1242
Epoch 5/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8625 - mse: 0.8600 - val_loss: 1.1587 - val_mse: 1.1559
Epoch 6/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8451 - mse: 0.8421 - val_loss: 1.1325 - val_mse: 1.1292
test MSE 1.0816
CrossNet parameterization: vector
Epoch 1/30
472/472 [==============================] - 14s 20ms/step - loss: 1.5136 - mse: 1.5134 - val_loss: 1.1219 - val_mse: 1.1213


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9668 - mse: 0.9659 - val_loss: 1.1050 - val_mse: 1.1039


Epoch 3/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9104 - mse: 0.9089 - val_loss: 1.1082 - val_mse: 1.1065
Epoch 4/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8772 - mse: 0.8752 - val_loss: 1.1151 - val_mse: 1.1129
Epoch 5/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8591 - mse: 0.8566 - val_loss: 1.1306 - val_mse: 1.1278
Epoch 6/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8423 - mse: 0.8393 - val_loss: 1.1472 - val_mse: 1.1439
Epoch 7/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8204 - mse: 0.8168 - val_loss: 1.1534 - val_mse: 1.1496
test MSE 1.0609
CrossNet parameterization: vector
Epoch 1/30
472/472 [==============================] - 13s 18ms/step - loss: 1.5579 - mse: 1.5576 - val_loss: 1.1179 - val_mse: 1.1173


Epoch 2/30
472/472 [==============================] - 9s 18ms/step - loss: 0.9633 - mse: 0.9624 - val_loss: 1.1043 - val_mse: 1.1031


Epoch 3/30
472/472 [==============================] - 9s 19ms/step - loss: 0.9042 - mse: 0.9027 - val_loss: 1.1413 - val_mse: 1.1396
Epoch 4/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8739 - mse: 0.8719 - val_loss: 1.1344 - val_mse: 1.1322
Epoch 5/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8577 - mse: 0.8552 - val_loss: 1.1529 - val_mse: 1.1501
Epoch 6/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8394 - mse: 0.8364 - val_loss: 1.1467 - val_mse: 1.1433
Epoch 7/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8159 - mse: 0.8123 - val_loss: 1.1511 - val_mse: 1.1472
test MSE 1.0797
CrossNet parameterization: vector
Epoch 1/30
472/472 [==============================] - 13s 18ms/step - loss: 1.5434 - mse: 1.5431 - val_loss: 1.1097 - val_mse: 1.1091


Epoch 2/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9687 - mse: 0.9678 - val_loss: 1.1014 - val_mse: 1.1002


Epoch 3/30
472/472 [==============================] - 9s 18ms/step - loss: 0.9083 - mse: 0.9068 - val_loss: 1.1113 - val_mse: 1.1096
Epoch 4/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8831 - mse: 0.8811 - val_loss: 1.1418 - val_mse: 1.1395
Epoch 5/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8653 - mse: 0.8628 - val_loss: 1.1144 - val_mse: 1.1116
Epoch 6/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8510 - mse: 0.8480 - val_loss: 1.1181 - val_mse: 1.1147
Epoch 7/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8380 - mse: 0.8344 - val_loss: 1.1438 - val_mse: 1.1399
test MSE 1.0702
CrossNet parameterization: vector
Epoch 1/30
472/472 [==============================] - 14s 19ms/step - loss: 1.5200 - mse: 1.5197 - val_loss: 1.1331 - val_mse: 1.1325


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9660 - mse: 0.9651 - val_loss: 1.1087 - val_mse: 1.1076


Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9058 - mse: 0.9044 - val_loss: 1.1514 - val_mse: 1.1497
Epoch 4/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8797 - mse: 0.8777 - val_loss: 1.1154 - val_mse: 1.1132
Epoch 5/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8597 - mse: 0.8572 - val_loss: 1.1427 - val_mse: 1.1400
Epoch 6/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8448 - mse: 0.8418 - val_loss: 1.1415 - val_mse: 1.1383
Epoch 7/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8259 - mse: 0.8224 - val_loss: 1.1325 - val_mse: 1.1287
test MSE 1.07


In [46]:
# NFM
dict_NFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                NFM, dict_NFM,
                                feature_names, 
                                'yelp', 'NFM')
all_result.extend(ls_result)

Epoch 1/30
472/472 [==============================] - 14s 19ms/step - loss: 1.7384 - mse: 1.7381 - val_loss: 1.1200 - val_mse: 1.1193


Epoch 2/30
472/472 [==============================] - 9s 19ms/step - loss: 0.9919 - mse: 0.9908 - val_loss: 1.0980 - val_mse: 1.0965


Epoch 3/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9282 - mse: 0.9263 - val_loss: 1.0924 - val_mse: 1.0901


Epoch 4/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8927 - mse: 0.8900 - val_loss: 1.1038 - val_mse: 1.1008
Epoch 5/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8671 - mse: 0.8638 - val_loss: 1.1340 - val_mse: 1.1303
Epoch 6/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8424 - mse: 0.8384 - val_loss: 1.1379 - val_mse: 1.1336
Epoch 7/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8214 - mse: 0.8167 - val_loss: 1.1626 - val_mse: 1.1577
Epoch 8/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8025 - mse: 0.7973 - val_loss: 1.1310 - val_mse: 1.1255
test MSE 1.0815
Epoch 1/30
472/472 [==============================] - 13s 18ms/step - loss: 1.7009 - mse: 1.7006 - val_loss: 1.1330 - val_mse: 1.1324


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9964 - mse: 0.9953 - val_loss: 1.0992 - val_mse: 1.0977


...

Epoch 4/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8971 - mse: 0.8945 - val_loss: 1.1099 - val_mse: 1.1069


Epoch 5/30
472/472 [==============================] - 8s 16ms/step - loss: 0.8781 - mse: 0.8747 - val_loss: 1.1041 - val_mse: 1.1004


Epoch 6/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8615 - mse: 0.8576 - val_loss: 1.1189 - val_mse: 1.1146
Epoch 7/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8484 - mse: 0.8438 - val_loss: 1.1047 - val_mse: 1.0999


Epoch 8/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8304 - mse: 0.8253 - val_loss: 1.1427 - val_mse: 1.1373
Epoch 9/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8147 - mse: 0.8090 - val_loss: 1.1343 - val_mse: 1.1283
Epoch 10/30
472/472 [==============================] - 8s 17ms/step - loss: 0.7976 - mse: 0.7913 - val_loss: 1.1262 - val_mse: 1.1197
Epoch 11/30
472/472 [==============================] - 8s 17ms/step - loss: 0.7827 - mse: 0.7760 - val_loss: 1.1409 - val_mse: 1.1339
Epoch 12/30
472/472 [==============================] - 8s 17ms/step - loss: 0.7655 - mse: 0.7584 - val_loss: 1.1682 - val_mse: 1.1608
test MSE 1.0922
Epoch 1/30
472/472 [==============================] - 13s 18ms/step - loss: 1.6669 - mse: 1.6666 - val_loss: 1.1062 - val_mse: 1.1056


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9820 - mse: 0.9810 - val_loss: 1.1362 - val_mse: 1.1349
Epoch 3/30
472/472 [==============================] - 9s 19ms/step - loss: 0.9163 - mse: 0.9146 - val_loss: 1.1137 - val_mse: 1.1118
Epoch 4/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8836 - mse: 0.8813 - val_loss: 1.1112 - val_mse: 1.1086
Epoch 5/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8615 - mse: 0.8586 - val_loss: 1.1241 - val_mse: 1.1209
Epoch 6/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8401 - mse: 0.8366 - val_loss: 1.1973 - val_mse: 1.1935
test MSE 1.0894


In [47]:
# deepFM
dict_deepFM = {'linear_feature_columns': linear_feature_columns, 
               'dnn_feature_columns': dnn_feature_columns,
               'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                DeepFM, dict_deepFM,
                                feature_names, 
                                'yelp', 'deepFM')
all_result.extend(ls_result)

Epoch 1/30
472/472 [==============================] - 12s 16ms/step - loss: 1.5630 - mse: 1.5628 - val_loss: 1.1367 - val_mse: 1.1362


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9683 - mse: 0.9676 - val_loss: 1.1050 - val_mse: 1.1040


Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9105 - mse: 0.9092 - val_loss: 1.1073 - val_mse: 1.1058
Epoch 4/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8807 - mse: 0.8789 - val_loss: 1.1536 - val_mse: 1.1516
Epoch 5/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8619 - mse: 0.8596 - val_loss: 1.1300 - val_mse: 1.1274
Epoch 6/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8396 - mse: 0.8368 - val_loss: 1.1312 - val_mse: 1.1281
Epoch 7/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8115 - mse: 0.8082 - val_loss: 1.1515 - val_mse: 1.1479
test MSE 1.0714
Epoch 1/30
472/472 [==============================] - 13s 18ms/step - loss: 1.5679 - mse: 1.5677 - val_loss: 1.1119 - val_mse: 1.1114


Epoch 2/30
472/472 [==============================] - 8s 18ms/step - loss: 0.9680 - mse: 0.9672 - val_loss: 1.0981 - val_mse: 1.0971


Epoch 3/30
472/472 [==============================] - 9s 18ms/step - loss: 0.9088 - mse: 0.9075 - val_loss: 1.1157 - val_mse: 1.1141
Epoch 4/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8811 - mse: 0.8793 - val_loss: 1.1231 - val_mse: 1.1210
Epoch 5/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8635 - mse: 0.8612 - val_loss: 1.1365 - val_mse: 1.1339
Epoch 6/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8468 - mse: 0.8439 - val_loss: 1.1257 - val_mse: 1.1225
Epoch 7/30
472/472 [==============================] - 9s 18ms/step - loss: 0.8271 - mse: 0.8237 - val_loss: 1.1685 - val_mse: 1.1648
test MSE 1.0568
Epoch 1/30
472/472 [==============================] - 15s 21ms/step - loss: 1.5313 - mse: 1.5311 - val_loss: 1.1235 - val_mse: 1.1230


...

Epoch 4/30
472/472 [==============================] - 9s 20ms/step - loss: 0.8758 - mse: 0.8740 - val_loss: 1.1401 - val_mse: 1.1381
Epoch 5/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8619 - mse: 0.8596 - val_loss: 1.1358 - val_mse: 1.1332
Epoch 6/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8444 - mse: 0.8416 - val_loss: 1.1547 - val_mse: 1.1516
Epoch 7/30
472/472 [==============================] - 9s 19ms/step - loss: 0.8238 - mse: 0.8205 - val_loss: 1.1357 - val_mse: 1.1321
Epoch 8/30
472/472 [==============================] - 9s 19ms/step - loss: 0.7989 - mse: 0.7950 - val_loss: 1.1591 - val_mse: 1.1550
test MSE 1.0861
Epoch 1/30
472/472 [==============================] - 13s 17ms/step - loss: 1.5456 - mse: 1.5453 - val_loss: 1.1020 - val_mse: 1.1015


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9694 - mse: 0.9687 - val_loss: 1.0859 - val_mse: 1.0849


Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9099 - mse: 0.9086 - val_loss: 1.1276 - val_mse: 1.1261
Epoch 4/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8821 - mse: 0.8803 - val_loss: 1.1056 - val_mse: 1.1036
Epoch 5/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8616 - mse: 0.8593 - val_loss: 1.1239 - val_mse: 1.1213
Epoch 6/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8384 - mse: 0.8356 - val_loss: 1.1494 - val_mse: 1.1463
Epoch 7/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8140 - mse: 0.8107 - val_loss: 1.1347 - val_mse: 1.1311
test MSE 1.063
Epoch 1/30
472/472 [==============================] - 14s 18ms/step - loss: 1.5179 - mse: 1.5177 - val_loss: 1.0970 - val_mse: 1.0965


Epoch 2/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9652 - mse: 0.9644 - val_loss: 1.0959 - val_mse: 1.0949


Epoch 3/30
472/472 [==============================] - 8s 17ms/step - loss: 0.9080 - mse: 0.9068 - val_loss: 1.1169 - val_mse: 1.1154
Epoch 4/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8816 - mse: 0.8799 - val_loss: 1.1655 - val_mse: 1.1635
Epoch 5/30
472/472 [==============================] - 8s 18ms/step - loss: 0.8618 - mse: 0.8595 - val_loss: 1.1192 - val_mse: 1.1167
Epoch 6/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8414 - mse: 0.8386 - val_loss: 1.1610 - val_mse: 1.1580
Epoch 7/30
472/472 [==============================] - 8s 17ms/step - loss: 0.8192 - mse: 0.8159 - val_loss: 1.1646 - val_mse: 1.1610
test MSE 1.0689


In [48]:
# AFM
dict_AFM = {'linear_feature_columns': linear_feature_columns,
            'dnn_feature_columns': dnn_feature_columns,
            'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                AFM, dict_AFM,
                                feature_names, 
                                'yelp', 'AFM')
all_result.extend(ls_result)

Epoch 1/30
472/472 [==============================] - 15s 19ms/step - loss: 2.6972 - mse: 2.6968 - val_loss: 1.2081 - val_mse: 1.2075


Epoch 2/30
472/472 [==============================] - 10s 22ms/step - loss: 1.2022 - mse: 1.2012 - val_loss: 1.1850 - val_mse: 1.1835


Epoch 3/30
472/472 [==============================] - 10s 22ms/step - loss: 1.1123 - mse: 1.1095 - val_loss: 1.1269 - val_mse: 1.1225


Epoch 4/30
472/472 [==============================] - 11s 22ms/step - loss: 0.9853 - mse: 0.9802 - val_loss: 1.1107 - val_mse: 1.1049


Epoch 5/30
472/472 [==============================] - 10s 22ms/step - loss: 0.9314 - mse: 0.9251 - val_loss: 1.1000 - val_mse: 1.0932


...

Epoch 8/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8702 - mse: 0.8600 - val_loss: 1.1172 - val_mse: 1.1066
Epoch 9/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8524 - mse: 0.8412 - val_loss: 1.1239 - val_mse: 1.1122
Epoch 10/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8363 - mse: 0.8241 - val_loss: 1.1263 - val_mse: 1.1136
Epoch 11/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8223 - mse: 0.8092 - val_loss: 1.1314 - val_mse: 1.1179
Epoch 12/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8088 - mse: 0.7949 - val_loss: 1.1524 - val_mse: 1.1380
test MSE 1.0754
Epoch 1/30
472/472 [==============================] - 16s 20ms/step - loss: 2.6696 - mse: 2.6691 - val_loss: 1.2046 - val_mse: 1.2038


Epoch 2/30
472/472 [==============================] - 9s 20ms/step - loss: 1.1986 - mse: 1.1973 - val_loss: 1.1813 - val_mse: 1.1791


Epoch 3/30
472/472 [==============================] - 9s 20ms/step - loss: 1.0754 - mse: 1.0714 - val_loss: 1.1234 - val_mse: 1.1181


Epoch 4/30
472/472 [==============================] - 10s 20ms/step - loss: 0.9669 - mse: 0.9608 - val_loss: 1.1076 - val_mse: 1.1009


Epoch 5/30
472/472 [==============================] - 9s 19ms/step - loss: 0.9215 - mse: 0.9141 - val_loss: 1.1205 - val_mse: 1.1126
Epoch 6/30
472/472 [==============================] - 9s 20ms/step - loss: 0.8956 - mse: 0.8872 - val_loss: 1.1247 - val_mse: 1.1159
Epoch 7/30
472/472 [==============================] - 9s 20ms/step - loss: 0.8780 - mse: 0.8688 - val_loss: 1.1209 - val_mse: 1.1114
Epoch 8/30
472/472 [==============================] - 9s 20ms/step - loss: 0.8648 - mse: 0.8548 - val_loss: 1.1299 - val_mse: 1.1196
Epoch 9/30
472/472 [==============================] - 10s 21ms/step - loss: 0.8555 - mse: 0.8449 - val_loss: 1.1346 - val_mse: 1.1237
test MSE 1.087


In [49]:
# xDeepFM
dict_xDeepFM = {'linear_feature_columns': linear_feature_columns,
                'dnn_feature_columns': dnn_feature_columns,
                'task': 'regression'}
ls_result = calc_nn_based_model(df_yelp_all, 
                                xDeepFM, dict_xDeepFM,
                                feature_names, 
                                'yelp', 'xDeepFM')
all_result.extend(ls_result)

Epoch 1/30
472/472 [==============================] - 14s 19ms/step - loss: 1.4801 - mse: 1.4799 - val_loss: 1.1049 - val_mse: 1.1044


Epoch 2/30
472/472 [==============================] - 10s 20ms/step - loss: 0.9649 - mse: 0.9641 - val_loss: 1.0953 - val_mse: 1.0943


Epoch 3/30
472/472 [==============================] - 9s 20ms/step - loss: 0.9071 - mse: 0.9058 - val_loss: 1.1011 - val_mse: 1.0996
Epoch 4/30
472/472 [==============================] - 9s 20ms/step - loss: 0.8782 - mse: 0.8764 - val_loss: 1.1280 - val_mse: 1.1259
Epoch 5/30
472/472 [==============================] - 9s 20ms/step - loss: 0.8545 - mse: 0.8521 - val_loss: 1.1320 - val_mse: 1.1294
Epoch 6/30
472/472 [==============================] - 10s 20ms/step - loss: 0.8213 - mse: 0.8185 - val_loss: 1.1632 - val_mse: 1.1601
Epoch 7/30
472/472 [==============================] - 9s 20ms/step - loss: 0.7835 - mse: 0.7802 - val_loss: 1.1517 - val_mse: 1.1481
test MSE 1.0675
Epoch 1/30
472/472 [==============================] - 17s 25ms/step - loss: 1.4701 - mse: 1.4698 - val_loss: 1.1285 - val_mse: 1.1280


Epoch 2/30
472/472 [==============================] - 10s 21ms/step - loss: 0.9668 - mse: 0.9660 - val_loss: 1.1045 - val_mse: 1.1035


Epoch 3/30
472/472 [==============================] - 10s 22ms/step - loss: 0.9074 - mse: 0.9061 - val_loss: 1.1101 - val_mse: 1.1085
Epoch 4/30
472/472 [==============================] - 10s 20ms/step - loss: 0.8775 - mse: 0.8757 - val_loss: 1.1140 - val_mse: 1.1119
Epoch 5/30
472/472 [==============================] - 10s 20ms/step - loss: 0.8540 - mse: 0.8517 - val_loss: 1.1339 - val_mse: 1.1313
Epoch 6/30
472/472 [==============================] - 10s 21ms/step - loss: 0.8269 - mse: 0.8240 - val_loss: 1.1440 - val_mse: 1.1409
Epoch 7/30
472/472 [==============================] - 10s 21ms/step - loss: 0.7916 - mse: 0.7883 - val_loss: 1.1498 - val_mse: 1.1462
test MSE 1.0581
Epoch 1/30
472/472 [==============================] - 16s 20ms/step - loss: 1.5002 - mse: 1.5000 - val_loss: 1.1070 - val_mse: 1.1065


...

Epoch 2/30
472/472 [==============================] - 10s 20ms/step - loss: 0.9662 - mse: 0.9654 - val_loss: 1.1061 - val_mse: 1.1050


Epoch 3/30
472/472 [==============================] - 9s 20ms/step - loss: 0.9089 - mse: 0.9076 - val_loss: 1.1050 - val_mse: 1.1035


Epoch 4/30
472/472 [==============================] - 9s 20ms/step - loss: 0.8807 - mse: 0.8788 - val_loss: 1.1059 - val_mse: 1.1038
Epoch 5/30
472/472 [==============================] - 10s 20ms/step - loss: 0.8610 - mse: 0.8587 - val_loss: 1.1111 - val_mse: 1.1085
Epoch 6/30
472/472 [==============================] - 9s 20ms/step - loss: 0.8412 - mse: 0.8383 - val_loss: 1.1089 - val_mse: 1.1057
Epoch 7/30
472/472 [==============================] - 9s 20ms/step - loss: 0.8122 - mse: 0.8088 - val_loss: 1.1368 - val_mse: 1.1331
Epoch 8/30
472/472 [==============================] - 9s 20ms/step - loss: 0.7753 - mse: 0.7713 - val_loss: 1.1526 - val_mse: 1.1484
test MSE 1.0799
Epoch 1/30
472/472 [==============================] - 16s 22ms/step - loss: 1.4909 - mse: 1.4906 - val_loss: 1.1175 - val_mse: 1.1170


Epoch 2/30
472/472 [==============================] - 11s 23ms/step - loss: 0.9656 - mse: 0.9648 - val_loss: 1.0950 - val_mse: 1.0940


Epoch 3/30
472/472 [==============================] - 10s 22ms/step - loss: 0.9056 - mse: 0.9043 - val_loss: 1.1197 - val_mse: 1.1181
Epoch 4/30
472/472 [==============================] - 10s 22ms/step - loss: 0.8772 - mse: 0.8754 - val_loss: 1.1171 - val_mse: 1.1150
Epoch 5/30
472/472 [==============================] - 11s 23ms/step - loss: 0.8516 - mse: 0.8493 - val_loss: 1.1397 - val_mse: 1.1371
Epoch 6/30
472/472 [==============================] - 10s 22ms/step - loss: 0.8232 - mse: 0.8204 - val_loss: 1.1669 - val_mse: 1.1638
Epoch 7/30
472/472 [==============================] - 11s 23ms/step - loss: 0.7895 - mse: 0.7861 - val_loss: 1.1385 - val_mse: 1.1348
test MSE 1.0737


In [50]:
pd.DataFrame(all_result, columns=['model', 'kfold', 'data', 'RMSE', 'Recall', 'NDCG']).to_csv('output/NN_result.csv', index=False)